In [ ]:
#import the data:

#first some constants:
Myr = 3.154e13
km = 1e3
kpc = 3.086e19

import pandas

#set number of particles:
np = 9
l = list(range(1,np+1))
#otherwise to load only data of specific particles, make l a list of those particles' numbers:
#l = [4]
#for surface of section runs:
p = 281
#l = [f'{p}'] + [f'{p}.{s}' for s in range(1,8)]
#for just a random subset of 20 particles:
#import random
#l = random.sample(range(1,np+1),400)

#for large runs with particles evenly spaced:

dat = {
    'r0': [],
    'phi0': []
}

r0 = [i/100 for i in range(60,305,5)]
phi0 = list(range(0,360,15))
dat['r0'] = [i for i in r0 for j in phi0]
dat['phi0'] = [j for i in r0 for j in phi0]

ps = pandas.DataFrame(dat,index=list(range(1,len(dat['r0'])+1)))
l = list(ps.index)

#print(l)
#pcols = [col for s in l for col in [f'n_{s}',f'x_{s}',f'y_{s}',f'vx_{s}',f'vy_{s}',f'ax_{s}',f'ay_{s}',f'PE_{s}']]
pcols = [col for s in l for col in [f'x_{s}',f'y_{s}',f'vx_{s}',f'vy_{s}',f'PE_{s}']]
cols = pcols + ['t', 'phib', 'omega', 'Rcr']
#print(cols)

nrows = int(10e3/0.1)
skiprows = [i for i in range(1,nrows+1) if i not in range(1,nrows+1,10)]

import time
t1 = time.time()
data = pandas.read_csv("C:/data10.txt", usecols=cols, header=0, skiprows=skiprows, comment='#')
t2 = time.time()
pandas.set_option('display.max_columns', 500)
display(data)
print(data.count())
print('time taken to read:',t2-t1)
print(data.info())

'''
datas = ['data10 - 3-16756e17','data10 - 4-75812e17','data10 - 6-15188e17']
for i in range(len(datas)):
    datas[i] = pandas.read_csv(f"C:/{datas[i]}.txt", usecols=cols, header=0, comment='#')
'''

In [ ]:
import numpy
t1 = time.time()

for s in l:
    #r = numpy.sqrt((data[f'x_{s}']**2) + (data[f'y_{s}']**2))
    #phi = numpy.arctan2(data[f'y_{s}'], data[f'x_{s}'])
    #phic = phi - data['phib']%(2*numpy.pi)
    data[f'xc_{s}'] = (data[f'x_{s}']*numpy.cos(data['phib'])) + (data[f'y_{s}']*numpy.sin(data['phib'])) #r*numpy.cos(phic)
    data[f'yc_{s}'] = (data[f'y_{s}']*numpy.cos(data['phib'])) - (data[f'x_{s}']*numpy.sin(data['phib'])) #r*numpy.sin(phic)
    #vr = ((data[f'vy_{s}']*data[f'y_{s}']) + (data[f'vx_{s}']*data[f'x_{s}']))/r
    #vphi = ((data[f'vy_{s}']*data[f'x_{s}']) - (data[f'vx_{s}']*data[f'y_{s}']))/r
    #phidot = vphi/r
    data[f'vxc_{s}'] = (data[f'vx_{s}']*numpy.cos(data['phib'])) + (data[f'vy_{s}']*numpy.sin(data['phib'])) #vr*numpy.cos(phic) - r*phidot*numpy.sin(phic)
    data[f'vyc_{s}'] = (data[f'vy_{s}']*numpy.cos(data['phib'])) - (data[f'vx_{s}']*numpy.sin(data['phib'])) #vr*numpy.sin(phic) + r*phidot*numpy.cos(phic)
    data[f'L_{s}'] = (data[f'vy_{s}']*data[f'x_{s}']) - (data[f'vx_{s}']*data[f'y_{s}'])
    data[f'Ej_{s}'] = (0.5*((data[f'vx_{s}']**2) + (data[f'vy_{s}']**2))) + data[f'PE_{s}'] - (data['omega']*data[f'L_{s}'])
t2 = time.time()
print('time taken:',t2-t1)
#data['2Rcr'] = 2*data['Rcr']
data['Rilr'] = (1-(1/numpy.sqrt(2)))*data['Rcr']
data['Rolr'] = (1+(1/numpy.sqrt(2)))*data['Rcr']
data.info()
#data[(data.index >= 0) & (data.index <= 20)]['r_3']

In [ ]:
data

In [ ]:
#identifying x1 orbits: #note: using scipy.signal.find_peaks might have been better to find maxima and minima, but as I'm just using shift=1 it doesn't really matter
apois = {}
#alternatively have a dataframe with a time column giving all the times of the simulation, and a column giving a list of particles that are on x1 orbits at that time

x1times = {}
tphi90s = {}
#x1indices = {}
#sizes = {}
shift = 1
withinphi = 40 #angle from the bar's major axis (in degrees) within which the apoapsis must be
max_ratio = 0.5 #the maximum ratio of periapsis to apoapsis when on an x1 orbit

def adjust(phi1,phi2): #this is to adjust the values of phi to their coterminal values with the minimum difference between each other, to be able to compare them correctly, so e.g. -170 degrees is actually a greater angle than 170 degrees, so adjust their values to either -170 and -190 or 190 and 170 before comparing with > or <
    #phi1 is adjusted by adding/subtracting 2pi until the difference between the two angles is the minimum, and then returned in adjusted form
    df = pandas.DataFrame({'minus2pi': phi1-(2*numpy.pi),
                        'nothing': phi1,
                        'plus2pi': phi1+(2*numpy.pi)})
    
    idx = numpy.argmin(abs(df.sub(phi2, axis=0)).values, axis=1)
    
    return pandas.Series(numpy.choose(idx, df.values.T), index=phi2.index)

time1 = time.time()
for s in l:
    phic = numpy.arctan2(data[f'yc_{s}'], data[f'xc_{s}'])
    tphi90 = data['t'][(adjust(phic,phic.shift(1)) >= numpy.pi/2) & (phic.shift(1) < numpy.pi/2)] #the times at which or just before which phic was equal to 90 degrees; implied here is that the particle is orbiting anticlockwise; using 'adjust' to avoid quirk when current phi is just below 180 degrees (2nd quadrant) and previous phi is above -180 degrees (3rd quadrant); adjusting phic instead of phic.shift(1) to avoid another quirk when previous phi is just below 90 degrees (1st quadrant) and current phi is just above -180 degrees in 3rd quadrant (which happens for orbits with very contracted central loop) -  notice all angles at positive x are <90 degrees but not angles at negative x are >90 degrees (only those at positive y are)
    tphi90s[s] = tphi90.index
    #print(tphi90.size)
    r = numpy.sqrt((data[f'xc_{s}']**2) + (data[f'yc_{s}']**2))
    aposr = r[(r > r.shift(shift)) & (r > r.shift(-shift))] #the local maxima in r when compared to its value 'shift' datapoints away
    #aposr = apos[apos.index.to_series() - apos.index.to_series().shift(1,fill_value=-shift) >= shift] #'shift' is the maximum length of a set of consecutive points in 'apos'; only keep points in 'apos' which are at least 'shift' away from the previous point; consider issues here with points at the beginning of 'apos'
    apois[s] = aposr.index
    #print(aposr.size)
    #tapos = data['t'][aposr.index]
    onx1t = {'from': [], 'to': []} #to contain the times between which a particle is on an x1 orbit
    #onx1i = [] #to contain the indices of all points of a particle when it is on an x1 orbit
    #sizes[s] = []
    for i in range(tphi90.size - 1):
        t1i = tphi90.index[i]
        t2i = tphi90.index[i+1]
        #1 azimuthal orbit is completed from t1 to t2, their indices being t1i and t2i
        n1 = numpy.arange(t1i,t2i+1)
        n2 = numpy.intersect1d(n1,aposr.index) #the indices of apoapses that occur between t1 and t2
        #sizes[s].append(n2.size)
        if n2.size == 2: #if the no. apoapses that occur between t1 and t2 is precisely 2
            condsa = []
            condsb = []
            for apoi in n2:
                phi = phic[apoi]
                #check if this apoapsis is within 'withinphi' degrees of the bar's major axis:
                cond1 = (phi < (withinphi/180)*numpy.pi) & (phi > (-withinphi/180)*numpy.pi)
                cond2 = (phi < ((-180+withinphi)/180)*numpy.pi)
                cond3 = (phi > ((180-withinphi)/180)*numpy.pi)
                if (cond1 | cond2 | cond3):
                    condsa.append(apoi)
                if aposr[apoi] < data['Rcr'][apoi]: #check it is within the corotation radius
                    condsb.append(apoi)
            if (len(condsa) == 2) and (len(condsb) == 2): #if both apopases are within 'withinphi' degrees of the bar's major axis and both are within Rcr
                minr = r[n1].min()
                rat = minr/max(aposr[n2[0]],aposr[n2[1]]) #ratio of minimum r to maximum r, i.e. b/a; which if you want can be converted into value for eccentricity as numpy.sqrt(1-((b/a)**2)) or ellipticity as 1-(b/a)
                if rat <= max_ratio:
                    onx1t['from'].append(tphi90[t1i])
                    onx1t['to'].append(tphi90[t2i])
                    #onx1i += list(range(t1i,t2i+1))
    x1times[s] = onx1t
    #x1indices[s] = onx1i
time2 = time.time()
print(time2-time1)
#print(x1times[270])
#print(sizes)
#print(aposr[aposr.index >= 9e3])
#print(tphi90[tphi90.index >= 9e3].index)

In [ ]:
#identifying consecutive x1 orbits:

x1timesc = {}
for s in l:
    x1timesc[s] = []
    if len(x1times[s]['from']) != 0:
        x1timesc[s].append([x1times[s]['from'][0],x1times[s]['to'][0]])
        for i in range(1,len(x1times[s]['from'])):
            if x1times[s]['from'][i] == x1timesc[s][-1][-1]:
                x1timesc[s][-1].append(x1times[s]['to'][i])
            else:
                x1timesc[s].append([x1times[s]['from'][i],x1times[s]['to'][i]])

print("The number of orbits in each particle's sequences of consecutive x1 orbits:")
lis1 = {}
for s in l:
    lis1[s] = [len(lis2)-1 for lis2 in x1timesc[s]]
    #lis1.append(len(x1timesc[s]))
print(lis1)

print('The particles with more than 1 sequence:')
lis2 = []
for s in l:
    if len(lis1[s]) > 1:
        lis2.append(s)
print(lis2)
#Each list in x1timesc[s] rpresents a sequence of consecutive x1 orbits. The elements are the times when the particle is at or was just at phi of 90 degrees, so the number of elements minus 1 is the number of consecutive orbits in that sequence, with it starting from the first element and ending at the last element

print('The particles which at some time in their trajectory are on x1 orbits:')
print([i for i in x1timesc if x1timesc[i]!=[]])

#for x1 in x1timesc[s]:
    #print("All", len(x1)-1, "of the particle's orbits from time", x1[0], "to time", x1[-1], "are x1 orbits")


In [ ]:
#converting into Gyr if you want:

x1timesc1 = {}
for s in x1timesc:
    x1timesc1[s] = []
    for i in range(len(x1timesc[s])):
        x1timesc1[s].append(numpy.array(x1timesc[s][i])/(Myr*1e3))
#print(x1timesc1[1])

x1timesc2 = {}
for s in x1timesc:
    x1timesc2[s] = numpy.array([numpy.array(el) for el in x1timesc[s]])/(Myr*1e3)
#print(x1timesc2[1])

In [ ]:
#checking particular orbit:

s = 281
shift = 1
withinphi = 40

print(x1timesc[s])

phic = numpy.arctan2(data[f'yc_{s}'], data[f'xc_{s}'])    
tphi90 = data['t'][(adjust(phic,phic.shift(1)) >= numpy.pi/2) & (phic.shift(1) < numpy.pi/2)] #the times at which or just before which phic was equal to 90 degrees; implied here is that the particle is orbiting anticlockwise; using 'adjust' to avoid quirk when current phi is just below 180 degrees (2nd quadrant) and previous phi is above -180 degrees (3rd quadrant); adjusting phic instead of phic.shift(1) to avoid another quirk when previous phi is just below 90 degrees (1st quadrant) and current phi is just above -180 degrees in 3rd quadrant (which happens for orbits with very contracted central loop) -  notice all angles at positive x are <90 degrees but not angles at negative x are >90 degrees (only those at positive y are)
#print(tphi90.size)
tsub = data['t'][(data['t'] >= 8.4*Myr*1e3) & (data['t'] <= 8.7*Myr*1e3)]
tphi90sub = tphi90[tphi90.index.intersection(tsub.index)]
print(tphi90sub)
r = numpy.sqrt((data[f'xc_{s}']**2) + (data[f'yc_{s}']**2))
aposr = r[(r > r.shift(shift)) & (r > r.shift(-shift))] #the local maxima in r when compared to its value 'shift' datapoints away
#aposr = apos[apos.index.to_series() - apos.index.to_series().shift(1,fill_value=-shift) >= shift] #'shift' is the maximum length of a set of consecutive points in 'apos'; only keep points in 'apos' which are at least 'shift' away from the previous point; consider issues here with points at the beginning of 'apos'
print(tsub[tsub.index.intersection(aposr.index)])
#print(aposr.size)
#tapos = data['t'][aposr.index]
#onx1t = {'from': [], 'to': []} #to contain the times between which a particle is on an x1 orbit
#onx1i = [] #to contain the indices of all points of a particle when it is on an x1 orbit
#sizes[s] = []
for i in range(tphi90sub.size - 1):
    t1i = tphi90sub.index[i]
    t2i = tphi90sub.index[i+1]
    print(t1i,t2i)
    print(tphi90sub[t1i],tphi90sub[t2i])
    #1 azimuthal orbit is completed from t1 to t2, their indices being t1i and t2i
    n1 = numpy.arange(t1i,t2i+1)
    n2 = numpy.intersect1d(n1,aposr.index) #the indices of apoapses that occur between t1 and t2
    print(n2)
    #sizes[s].append(n2.size)
    if n2.size == 2: #if the no. apoapses that occur between t1 and t2 is precisely 2
        condsa = []
        condsb = []
        for apoi in n2:
            apo = aposr[apoi]
            #check if this apoapsis is within 'withinphi' degrees of the bar's major axis:
            cond1 = (apo < (withinphi/180)*numpy.pi) & (apo > (-withinphi/180)*numpy.pi)
            cond2 = (apo < (-180+withinphi/180)*numpy.pi)
            cond3 = (apo > (180-withinphi/180)*numpy.pi)
            if (cond1 | cond2 | cond3):
                condsa.append(True)
            if apo < data['Rcr'][apoi]: #check it is within the corotation radius
                condsb.append(True)
        print(condsa,condsb)
        if (len(condsa) == 2) and (len(condsb) == 2):
            minr = r[n1].min()
            rat = minr/max(aposr[n2[0]],aposr[n2[1]])
            print(minr/max(aposr[n2[0]],aposr[n2[1]]),)
            if ecc <= 0.5:
                pass
                #onx1t['from'].append(tphi90[t1i])
                #onx1t['to'].append(tphi90[t2i])
                #onx1i += list(range(t1i,t2i+1))


In [ ]:
#checking how far the sequences of x1 orbits extends:

lis1 = [i for i in l if x1timesc[i] != []]
lis2 = [i for i in lis1 if x1timesc[i][-1][-1] > 9.9*(Myr*1e3)]
print(len(lis2))

print([i for i in lis1 if i not in lis2])

In [ ]:
#finding which x1 criteria aren't fulfilled in the orbit just before the first x1 orbit:

fcriteria = {
    1: [],
    2: [],
    3: [],
    4: []
}

for s in [i for i in l if x1timesc[i]!=[]]:
    t1i = tphi90s[s][data['t'][tphi90s[s]] < x1times[s]['from'][0]][-1]
    t2i = tphi90s[s][data['t'][tphi90s[s]] == x1times[s]['from'][0]][0]
    n1 = numpy.arange(t1i,t2i+1)
    n2 = numpy.intersect1d(n1,apois[s]) #the indices of apoapses that occur between t1 and t2
    #sizes[s].append(n2.size)
    if n2.size != 2: #if the no. apoapses that occur between t1 and t2 is precisely 2
        fcriteria[1].append(s)
    condsa = []
    condsb = []
    phic = numpy.arctan2(data[f'yc_{s}'], data[f'xc_{s}'])
    r = numpy.sqrt((data[f'xc_{s}']**2) + (data[f'yc_{s}']**2))
    for apoi in n2:
        phi = phic[apoi]
        #check if this apoapsis is within 'withinphi' degrees of the bar's major axis:
        cond1 = (phi < (withinphi/180)*numpy.pi) & (phi > (-withinphi/180)*numpy.pi)
        cond2 = (phi < ((-180+withinphi)/180)*numpy.pi)
        cond3 = (phi > ((180-withinphi)/180)*numpy.pi)
        if (cond1 | cond2 | cond3):
            condsa.append(apoi)
        if r[apoi] < data['Rcr'][apoi]: #check it is within the corotation radius
            condsb.append(apoi)
    if len(condsa) != 2:
        fcriteria[2].append(s)
    if len(condsb) != 2:
        fcriteria[3].append(s)
    minr = r[n1].min()
    rat = minr/max(r[n2[0]],r[n2[1]]) #ratio of minimum r to maximum r, i.e. b/a; which if you want can be converted into value for eccentricity as numpy.sqrt(1-((b/a)**2)) or ellipticity as 1-(b/a)
    if rat > max_ratio:
        fcriteria[4].append(s)


In [ ]:
#finding ratio of minima in r to maxima
shift = 1

maxrs = {}
minrs = {}
rats = pandas.DataFrame(index = data.index)
t1 = time.time()
for s in l:
    r = numpy.sqrt((data[f'xc_{s}']**2) + (data[f'yc_{s}']**2))
    maximarr = r[(r > r.shift(shift)) & (r > r.shift(-shift))]
    #maximary = maxima[maxima.index.to_series() - maxima.index.to_series().shift(1,fill_value=-shift) >= shift]
    maxrs[s] = maximarr.index
    minimarr = r[(r < r.shift(shift)) & (r < r.shift(-shift))]
    #maximarx = maxima[maxima.index.to_series() - maxima.index.to_series().shift(1,fill_value=-shift) >= shift]
    minrs[s] = minimarr.index
    maximarri = maximarr.reindex(minimarr.index, method='nearest')
    rats[f'rat_{s}'] = minimarr/maximarri
t2 = time.time()
print(t2-t1)

In [ ]:
#finding maxima in absolute value of yc:
shift = 1

rats = pandas.DataFrame(index = data.index)
t1 = time.time()
for s in l:
    r = numpy.sqrt((data[f'xc_{s}']**2) + (data[f'yc_{s}']**2))
    for i in range(tphi90s[s].size - 1):
        t1i = tphi90s[s][i]
        t2i = tphi90s[s][i+1]
        n = numpy.arange(t1i,t2i+1)
        rat = r[n].min()/r[n].max()
        idx = (t1i+t2i)//2
        rats.at[idx, f'rat_{s}'] = rat
t2 = time.time()
print(t2-t1)

In [ ]:
#for those with more than 1 sequence of x1 orbits, see which ones violated the max_ratio condition after first going onto x1 orbits:

for s in [i for i in l if x1timesc[i]!=[]]:
    sub = rats[f'rat_{s}'][rats[f'rat_{s}'].notna()]
    sub1 = sub[data['t'] >= x1timesc[s][0][0]]
    subbti = sub1.index[sub1 <= max_ratio][0]
    sub2 = sub1[sub1.index >= subbti]
    sub3 = sub2[sub2 > max_ratio]
    if sub3.empty:
        pass
    else:
        print(sub3)

In [ ]:
#experimenting with Fourier analysis:

from bokeh.plotting import figure, show, save
from bokeh.models import Legend, ColumnDataSource
from bokeh.io import output_notebook, reset_output

#try e.g. 281 for x1 orbit, 649 for rectangular orbit
s = 270
cond = (data['t'] > Myr*9e3) & (data['t'] < Myr*10e3)

r = numpy.sqrt((data[f'x_{s}']**2) + (data[f'y_{s}']**2))
phic = numpy.arctan2(data[f'yc_{s}'], data[f'xc_{s}'])
#print(min(phic))
#print(len(phic[phic>0]))
#print(phic[phic.index >= 3687])
#print(phic[phic.index > phic.index[phic == max(phic)][0]])

rcof = numpy.fft.rfft(r[cond])
rmax = numpy.argmax(abs(rcof[1:])) + 1
phicof = numpy.fft.rfft(phic[cond])
phimax = numpy.argmax(abs(phicof[1:])) + 1
print('No. radial oscillations per orbit =', rmax/phimax)
print('period of radial motion =', r[cond].size/rmax, 'Myr')
print('period of azimuthal motion =', phic[cond].size/phimax, 'Myr')
TOOLTIPS = [
    ("index", "$index")#,
#    ("name", "$name"),
#    ("(x,y)", "($x, $y)")
]

#p = figure(title="other", x_axis_label='x axis', y_axis_label='y axis', sizing_mode = 'stretch_both', tools = "pan,ywheel_pan,tap,box_zoom,wheel_zoom,zoom_in,zoom_out,undo,redo,reset,save", tooltips = TOOLTIPS)#, match_aspect=True, aspect_scale=1e-17) #lasso_select,box_select don't seem to work with the lines

p1 = figure(sizing_mode = 'stretch_both', tools = "pan,ywheel_pan,tap,box_zoom,wheel_zoom,zoom_in,zoom_out,undo,redo,reset,save", tooltips = TOOLTIPS)
p2 = figure(sizing_mode = 'stretch_both', tools = "pan,ywheel_pan,tap,box_zoom,wheel_zoom,zoom_in,zoom_out,undo,redo,reset,save", tooltips = TOOLTIPS)
p1.circle(x = list(range(len(rcof))), y = abs(rcof))
p2.circle(x = list(range(len(phicof))), y = abs(phicof))
output_notebook()
show(p1)
show(p2)
reset_output()

#the time period of radial motion is therefore expected to be the time length of the array r[cond] divded by rmax, and likewise for azimuthal motion

In [ ]:
#testing further:

wsize = 500

for s in ps.index[ps['r0'] == 0.65]:
    r = numpy.sqrt((data[f'x_{s}']**2) + (data[f'y_{s}']**2))
    phic = numpy.arctan2(data[f'yc_{s}'], data[f'xc_{s}'])
    freqs = []
    for i in range(data.index[wsize], data.index[-wsize]):
        cond = (data.index >= i-wsize) & (data.index <= i+wsize)
        rcof = numpy.fft.rfft(r[cond])
        rmax = numpy.argmax(abs(rcof[1:])) + 1
        phicof = numpy.fft.rfft(phic[cond])
        phimax = numpy.argmax(abs(phicof[1:])) + 1
        freqs.append(rmax/phimax)
    print([freqs[j] for j in range(len(freqs)) if j%1000==0])

In [ ]:
#condition for CR orbit: max and min phi be within particular angle of each other

t1 = time.time()
cr = {}

def adjust(phi1,phi2): #this is to adjust the values of phi to their coterminal values with the minimum difference between each other, to be able to compare them correctly, so e.g. -170 degrees is actually a greater angle than 170 degrees, so adjust their values to either -170 and -190 or 190 and 170 before comparing with > or <
    #phi1 is adjusted by adding/subtracting 2pi until the difference between the two angles is the minimum, and then returned in adjusted form
    df = pandas.DataFrame({'minus2pi': phi1-(2*numpy.pi),
                        'nothing': phi1,
                        'plus2pi': phi1+(2*numpy.pi)})
    
    idx = numpy.argmin(abs(df.sub(phi2, axis=0)).values, axis=1)
    
    return pandas.Series(numpy.choose(idx, df.values.T), index=phi2.index)

shift = 10 #the number of rows away from the previous/next angle to compare with; to mitigate the issue of local minima and maxima from zigzags

for s in l[300:320]:
    #t11 = time.time()
    phic = numpy.arctan2(data[f'yc_{s}'], data[f'xc_{s}'])
    #setting variables for the series' of angles before and after the current angle, and truncating the series' to get rid of the Nans that appear either end as a result of shifting:
    phicb = phic.shift(shift).dropna() #the angles just before the current angle
    phicbt = phic.truncate(shift) #just adjusting the size of phic to be the same before comparing with the former
    phica = phic.shift(-shift).dropna() #the angles just after the current angle
    phicat = phic.truncate(after=phic.index[-1]-shift) #just adjusting the size of phic to be the same before comparing with the former
    #print(phicb.index[0],phicb.index[-1],phicbt.index[0],phicbt.index[-1])
    #minphi = data1[adjust(phicbt,phicb) <= phicb]
    minphis = phic[(adjust(phicbt,phicb) <= phicb) & (adjust(phicat,phica) <= phica)] #local minima in phi
    maxphis = phic[(adjust(phicbt,phicb) >= phicb) & (adjust(phicat,phica) >= phica)] #local maxima in phi
    maxphisn = maxphis.reindex(minphis.index,method='bfill') #the subsequent maximum in phi of each minimum in phi, given the same indices for comparison
    #minphisn = minphis.reindex(maxphis.index,method='bfill')
    sub = minphis[(maxphisn-minphis)%(2*numpy.pi) < numpy.pi] #the difference between the two phis should be less than a particular value
    #NOTE: I PREVIOUSLY HAD USED abs HERE, BUT I DON'T THINK IT SHOULD BE USED HERE; CONSIDER E.G. A MINIMUM OF -10 DEGREES FOLLOWED BY A MAXIMUM OF -170 DEGREES, WHERE THE DIFFERENCE SHOULD BE 200 DEGREES NOT 160; LIKEWISE E.G. A MINIMUM OF 140 DEGREES FOLLOWED BY A MAXIMUM OF -150 DEGREES
    #print(sub.index[0],sub.index[-1])
    if sub.index.empty:
        cr[s] = pandas.Index([-1]) #just so as not to result in issues when trying to select the first and last index from cr[s] subseqeuntly
    else:
        cr[s] = sub.index #the indices of the minima in phi which, together with their subsequent maxima, fulfill the above condition
    
t3 = time.time()
print(t3-t1)
print(cr)
#data.loc[apoapses[3]]

In [ ]:
#alternative condition for CR orbits: max and min phi be within particular range

t1 = time.time()
cr = {}

def adjust(phi1,phi2): #this is to adjust the values of phi to their coterminal values with the minimum difference between each other, to be able to compare them correctly, so e.g. -170 degrees is actually a greater angle than 170 degrees, so adjust their values to either -170 and -190 or 190 and 170 before comparing with > or <
    #phi1 is adjusted by adding/subtracting 2pi until the difference between the two angles is the minimum, and then returned in adjusted form
    df = pandas.DataFrame({'minus2pi': phi1-(2*numpy.pi),
                        'nothing': phi1,
                        'plus2pi': phi1+(2*numpy.pi)})
    
    idx = numpy.argmin(abs(df.sub(phi2, axis=0)).values, axis=1)
    
    return pandas.Series(numpy.choose(idx, df.values.T), index=phi2.index)

shift = 10 #the number of rows away from the previous/next angle to compare with; to mitigate the issue of local minima and maxima from zigzags

for s in l:
    #t11 = time.time()
    phic = numpy.arctan2(data[f'yc_{s}'], data[f'xc_{s}'])
    #setting variables for the series' of angles before and after the current angle, and truncating the series' to get rid of the Nans that appear either end as a result of shifting:
    phicb = phic.shift(shift).dropna() #the angles just before the current angle
    phicbt = phic.truncate(shift) #just adjusting the size of phic to be the same before comparing with the former
    phica = phic.shift(-shift).dropna() #the angles just after the current angle
    phicat = phic.truncate(after=phic.index[-1]-shift) #just adjusting the size of phic to be the same before comparing with the former
    #print(phicb.index[0],phicb.index[-1],phicbt.index[0],phicbt.index[-1])
    #minphi = data1[adjust(phicbt,phicb) <= phicb]
    minphis = phic[(adjust(phicbt,phicb) <= phicb) & (adjust(phicat,phica) <= phica)] #local minima in phi
    maxphis = phic[(adjust(phicbt,phicb) >= phicb) & (adjust(phicat,phica) >= phica)] #local maxima in phi
    maxphisn = maxphis.reindex(minphis.index,method='bfill') #the subsequent maximum in phi of each minimum in phi, given the same indices for comparison
    #minphisn = minphis.reindex(maxphis.index,method='bfill')
    #stipulate that a minimum in phi and its subsequent maximum be within a particular range or particular ranges of phi:
    cond1 = (maxphisn < numpy.pi*(8/9)) & (maxphisn > numpy.pi/9) & (minphis > numpy.pi/9) & (minphis < numpy.pi*(8/9))
    cond2 = (maxphisn < -numpy.pi/9) & (maxphisn > -numpy.pi*(8/9)) & (minphis > -numpy.pi*(8/9)) & (minphis < -numpy.pi/9)
    sub = minphis[cond1 | cond2]
    #print(sub.index[0],sub.index[-1])
    if sub.index.empty:
        cr[s] = pandas.Index([-1]) #just so as not to result in issues when trying to select the first and last index from cr[s] subseqeuntly
    else:
        cr[s] = sub.index #the indices of the minima in phi which, together with their subsequent maxima, fulfill the above condition
    
t3 = time.time()
print(t3-t1)
print(cr)
#data.loc[apoapses[3]]

In [ ]:
#for surfaces of section:

pl = l#[i for i in ps.index[ps['r0'] == 0.6] if x1timesc[i]!=[]]#l #[1,2,3,6,7,8,11,15] #a list of the particles to plot
visible_lines = l#[pl[0]] #a list of the particles the lines of which are to be made initially visible on the plot; particles not in pl have no effect
#if you want to make the lines of all particles visible except some particles s1,s2,s3 then use:
#visible_lines = [s for s in l if s not in {s1,s2,s3}]

from bokeh.plotting import figure, show, save
from bokeh.models import Legend, ColumnDataSource

TOOLTIPS = [
    ("time", "@t"),
    ("name", "$name"),
    ("(x,y)", "($x, $y)")
]

p = figure(title="other", x_axis_label='x axis', y_axis_label='y axis', sizing_mode = 'stretch_both', tools = "pan,ywheel_pan,tap,box_zoom,wheel_zoom,zoom_in,zoom_out,undo,redo,reset,save", tooltips = TOOLTIPS)#, match_aspect=True, aspect_scale=1e-17) #lasso_select,box_select don't seem to work with the lines
#l = ['']
#for i in range(2,20):
#    l.append('.{}'.format(i))
#print(l)

#bokeh doesn't automatically cycle through colours for different lines; see: https://stackoverflow.com/questions/39839409/when-plotting-with-bokeh-how-do-you-automatically-cycle-through-a-color-pallett, https://docs.bokeh.org/en/latest/docs/reference/palettes.html, 
from bokeh.palettes import Category20b
import itertools
colours = itertools.cycle(Category20b[20]) #cycle back through list when it reaches end
#if you want to plot more than 20 with different colours, then use (for up to 256):
#import all_pallettes
#colours = all_palettes['Turbo'][256][0:256-(256%np):(256//np)] #to give colours equally spaced out in that spectrum
#...
#legend_items1 = [legend_items[i:i+20] for i in range(0, len(legend_items), 20)] #to make multiple legends, as 1 legend gets truncated
#for item in legend_items1:
#    legend = Legend(items = item)

legend_items = [] #solution for legend based on: https://stackoverflow.com/questions/46730609/position-the-legend-outside-the-plot-area-with-bokeh

#source = ColumnDataSource(data=dict())

for s,colour in zip(pl,colours):
    #data1 = data[data.index < min(idxs1)] #only plot up to when any particle first changes n
    #data1 = data[data.index < idxs1[s-1]] #for each particle, only plot up to when it first changes n (if it changes)
    #data1 = data[data.index < 20]
    #then change references to 'data' below to 'data1':
    #x = data[f't']
    #y = data[f'r_{s}']
    #plot = p.line(x, y, line_color = colour, line_width=2, name = f'p_{s}', visible=False, selection_alpha = 0.2) #wasn't able to mute all lines except selected, so set it to mute selected line. Use Ctrl to select multiple lines and click on no line to unselect
    
    #data1 = data[(data[f'r_{s}'] > data[f'r_{s}'].shift(1)) & (data[f'r_{s}'] > data[f'r_{s}'].shift(-1))]
    #x1 = data1[f't']
    #y1 = data1[f'r_{s}']
    #plot1 = p.circle(x1, y1, color='yellow', size=7)
    
    sub = data[(data[f'xc_{s}'] > 0) & (data[f'xc_{s}'].shift(1) < 0)]
    sub = sub[(sub['t'] > Myr*0e3) & (sub['t'] < Myr*30e3)] #sub[(sub.index > 30000) & (sub.index < 40000)]
    #sub = sub[sub[f'yc_{s}'] > -9e19]
    #sub = sub[data[f'xc_{s}'].shift(1) > data[f'xc_{s}'].shift(2)]
    #print(sub.shape)
    #x = []
    #y = []
    source = ColumnDataSource(data={'x':[],'y':[],'t':[]})
    for i in sub.index:
        xval = numpy.interp(0,[data[f'xc_{s}'].loc[i-1],data[f'xc_{s}'].loc[i]],[data[f'yc_{s}'].loc[i-1],data[f'yc_{s}'].loc[i]])
        yval = numpy.interp(0,[data[f'xc_{s}'].loc[i-1],data[f'xc_{s}'].loc[i]],[data[f'vyc_{s}'].loc[i-1],data[f'vyc_{s}'].loc[i]])
        tval = data[f't'].loc[i-1] #the time of the last row when it was at negative x
        source.data['x'].append(xval)
        source.data['y'].append(yval)
        source.data['t'].append(f'{tval:e}')
    plot = p.circle('x','y',source=source,color=colour,visible=False,name=f'p_{s}')
    #plot = p.dot(x, y, name = f'p_{s}', visible=False, selection_alpha = 0.2)
    #y2 = data[f'Ej_{s}']
    #plot1 = p.line(x, y2, line_color = colour, line_width=2, name = f'Ej_{s}', visible=False)
    if (s in visible_lines) or (f'{s}' in visible_lines):
        plot.visible = True
        #plot1.visible = True
    legend_items.append([f'particle {s}',[plot]])

if len(legend_items) <= 24: #only show legend if plotting 20 particles or less
    legend = Legend(items = legend_items)
    legend.click_policy="hide"
    p.add_layout(legend, 'right')

#show(p)
save(p,'plot2.html')

In [ ]:
#for making more particles for surfaces of section:

tval = 6.15188e17
p = 281
t_end = (3.154e13)*30e3

#Af = 0.02
#xi = 2*(tval/(Myr*2e3)) - 1
#A = Af*(0.1875*(xi**5) - 0.625*(xi**3) + 0.9375*xi + 0.5)

sub = data[data['t'] == tval]
#cols = ['t','phib','omega','Rcr'] + [col for col in [f'xc_{p}',f'yc_{p}',f'vxc_{p}',f'vyc_{p}']]
#sub1 = data[data.index >= sub.index[0]]
#display(sub1[cols])

f = list(i/10 for i in range(7,15) if i != 10)

lp = [f'{p}.{s}' for s in range(1,len(f)+1)]
lp += [f'{p}']
print(lp)

for i in range(len(f)):
    vyc = f[i]*sub[f'vyc_{p}']
    b = sub['omega']*sub[f'yc_{p}']
    vxc = -b - numpy.sqrt((b**2) - (vyc**2) - (2*sub[f'PE_{p}']) + (2*sub['omega']*sub[f'xc_{p}']*vyc) + (2*sub[f'Ej_{p}']))
    #print(vxc.iat[0])
    sub[f'x_{lp[i]}'] = sub[f'x_{p}']
    sub[f'y_{lp[i]}'] = sub[f'y_{p}']
    sub[f'vx_{lp[i]}'] = (vxc.iat[0]*numpy.cos(sub['phib'])) - (vyc.iat[0]*numpy.sin(sub['phib']))
    sub[f'vy_{lp[i]}'] = (vxc.iat[0]*numpy.sin(sub['phib'])) + (vyc.iat[0]*numpy.cos(sub['phib']))

cols = [col for s in lp for col in [f'x_{s}',f'y_{s}',f'vx_{s}',f'vy_{s}']]
cols = ['t','tend','phib','omega','Rcr'] + cols
sub['tend'] = t_end
#sub['A'] = A
sub.to_csv("C:/init.txt", columns=cols, index=False)


In [ ]:
tval = 3.154e17
t_end = (3.154e13)*20e3

sub = data[data['t'] == tval]

lp = [i for i in x1timesc if x1timesc[i]!=[]]

cols = [col for s in lp for col in [f'x_{s}',f'y_{s}',f'vx_{s}',f'vy_{s}']]
cols = ['t','tend','phib','omega','Rcr'] + cols
sub['tend'] = t_end
#sub['A'] = A
sub.to_csv("C:/init.txt", columns=cols, index=False)


In [ ]:
#for other plots:

pl = ps.index[(ps['r0'] == 1.05) & (ps['phi0'] == 0)]#[i for i in ps.index[(ps['r0'] <= 1.25)] if x1timesc[i] != []] #a list of the particles to plot
visible_lines = l#[pl[0]] #a list of the particles the lines of which are to be made initially visible on the plot; particles not in pl have no effect
#if you want to make the lines of all particles visible except some particles s1,s2,s3 then use:
#visible_lines = [s for s in l if s not in {s1,s2,s3}]

from bokeh.plotting import figure, show, save
from bokeh.models import Legend, ColumnDataSource, HoverTool, Span

tool = HoverTool(
    tooltips = [
        ("time", "@t"),#@t{%e}
        ("name", "$name"),
        ("(x,y)", "($x, $y)")
    ],
    formatters = {"@t": "printf"}
)

p = figure(title="other", x_axis_label='x axis', y_axis_label='y axis', sizing_mode = 'stretch_both', tools = "pan,ywheel_pan,tap,box_zoom,wheel_zoom,zoom_in,zoom_out,undo,redo,reset,save", match_aspect=True, aspect_scale=1e0) #lasso_select,box_select don't seem to work with the lines
p.add_tools(tool)

#l = ['']
#for i in range(2,20):
#    l.append('.{}'.format(i))
#print(l)

#bokeh doesn't automatically cycle through colours for different lines; see: https://stackoverflow.com/questions/39839409/when-plotting-with-bokeh-how-do-you-automatically-cycle-through-a-color-pallett, https://docs.bokeh.org/en/latest/docs/reference/palettes.html, 
from bokeh.palettes import Category20b
import itertools
colours = itertools.cycle(Category20b[20]) #cycle back through list when it reaches end
#if you want to plot more than 20 with different colours, then use (for up to 256):
#import all_pallettes
#colours = all_palettes['Turbo'][256][0:256-(256%np):(256//np)] #to give colours equally spaced out in that spectrum
#...
#legend_items1 = [legend_items[i:i+20] for i in range(0, len(legend_items), 20)] #to make multiple legends, as 1 legend gets truncated
#for item in legend_items1:
#    legend = Legend(items = item)

legend_items = [] #solution for legend based on: https://stackoverflow.com/questions/46730609/position-the-legend-outside-the-plot-area-with-bokeh

#Rcr0 = data['Rcr'][0]

plotted = []

pd = {}
for s,colour in zip(pl,colours):
    plots = []
    
    #to plot the particle's x1 orbits (using either x1timesc or x1times; use 1 of the 2 for loops):
    #for i in range(len(x1timesc[s])): #x1times[s]['from']
        #cond = (data['t'] >= x1timesc[s][i][0]) & (data['t'] <= x1timesc[s][i][-1])
    #for i in x1times[s]['from']:
        #cond = (data['t'] >= x1times[s]['from'][i]) & (data['t'] <= x1times[s]['to'][i])
    #then indent lines up to 'plots.append(...' below into the chosen for loop
    
    #to plot the trajectory of particles when their not on x1 orbits (in which case use pl = [i for i in x1timesc if x1timesc[i]!=[]]):
    #cond = (data['t'] <= x1timesc[s][0][0]) #before it's first x1 orbit
    #for i in range(len(x1timesc[s]) - 1): #x1times[s]['from']
        #cond = cond | ((data['t'] >= x1timesc[s][i][-1]) & (data['t'] <= x1timesc[s][i+1][0])) #between sequences of x1 orbits
    #cond = cond | (data['t'] >= x1timesc[s][-1][-1]) #after its last sequence of x1 orbits
    
    cond = (data['t'] >= Myr*4e3) & (data['t'] <= Myr*10e3) #(0.8*ps.at[s,'r0'] <= data['Rcr']/data['Rcr'][0]) & (1.2*ps.at[s,'r0'] >= data['Rcr']/data['Rcr'][0]) #(data.index >= cr[s][0]) & (data.index <= cr[s][-1]) #data.index > -1
    
    #r = numpy.sqrt((data[f'xc_{s}']**2) + (data[f'yc_{s}']**2))
    #apos = r[(r > r.shift(shift)) & (r > r.shift(-shift))][cond]
    #apoisscipy = r[find_peaks(r)[0]][cond]
    
    #omk2 = (1-(1/numpy.sqrt(2)))*235*km/(ps['r0'].at[s] * data['Rcr'][0])
    
    pd['x'] = data[f'xc_{s}'][cond]
    pd['y'] = data[f'yc_{s}'][cond]
    pd['t'] = data['t'][cond]/(Myr*1e3)
    #pd['r'] = r[cond]#*1e5
    
    source = ColumnDataSource(pd)
    
    plots.append(p.line('x', 'y', line_color = colour, line_width=2, name = f'p_{s}', visible=False, selection_alpha = 0.2, source=source)) #wasn't able to mute all lines except selected, so set it to mute selected line. Use Ctrl to select multiple lines and click on no line to unselect
    #plots.append(p.line('t', 'r', line_color = colour, line_width=2, name = f'p_{s}', visible=False, selection_alpha = 0.2, source=source)) #wasn't able to mute all lines except selected, so set it to mute selected line. Use Ctrl to select multiple lines and click on no line to unselect
    #data2 = data1[(data1[f'xc_{s}'] < data1[f'xc_{s}'].shift(1)) & (data1[f'xc_{s}'] < data1[f'xc_{s}'].shift(-1))]
    #data2 = data1.loc[data1.index.intersection(cr[s])]
    #x1 = data[f'xc_{s}'][aposr.index][cond]
    #y1 = data[f'yc_{s}'][aposr.index][cond]
    #x2 = data[f'xc_{s}'][tphi90.index][cond]
    #y2 = data[f'yc_{s}'][tphi90.index][cond]
    #plot1 = p.circle(x1, y1, color='yellow', size=7, visible=False)
    #plot2 = p.circle(x2, y2, color='blue', size=7, visible=False)
    #plots.append(p.circle(data['t'][apois[s]]/(Myr*1e3), r[apois[s]]*1e5, color='yellow'))
    
    #plots.append(p.circle(pd['x'][apos.index], pd['y'][apos.index], color='yellow', size = 7, visible=False))
    #plots.append(p.circle(pd['x'][apoisscipy.index], pd['y'][apoisscipy.index], color='red', size = 7, visible=False))
    
    #ts = x1timesc[s][0][0]
    #Lts = data[f'L_{s}'][data['t'] == ts].iat[0]
    #plots.append(p.circle(ts/(Myr*1e3),Lts, color = 'yellow', size=7))
    #span = Span(location = ts/(Myr*1e3), dimension = 'height')
    #p.add_layout(span)
    
    #plot = p.dot(x, y, name = f'p_{s}', visible=False, selection_alpha = 0.2)
    #y2 = data[f'Ej_{s}']
    #plot1 = p.line(x, y2, line_color = colour, line_width=2, name = f'Ej_{s}', visible=False)
    
    #if ps['r0'][s] not in plotted:
    #    span = Span(location = omk2, dimension = 'width')
    #    p.add_layout(span)
    #    plotted.append(ps['r0'][s])
    
    #plots = [plot]#,plot1,plot2]
    
    if (s in visible_lines) or (f'{s}' in visible_lines):
        for plot in plots:
            plot.visible = True
    legend_items.append([f'particle {s}',plots])

if len(legend_items) <= 24: #only show legend if plotting 20 particles or less
    legend = Legend(items = legend_items)
    legend.click_policy="hide"
    p.add_layout(legend, 'right')

#ILR = p.line(pd['t'], 235*km*data['Rilr'], name = 'ILR')
#OLR = p.line(pd['t'], 235*km*data['Rolr'], name = 'OLR')
#CR = p.line(pd['t'], 235*km*data['Rcr'], name = 'CR')
#omk2_bar = p.line(pd['t'], (1-(1/numpy.sqrt(2)))*data['omega'], name = 'omk2_bar')
#omega = p.line(pd['t'], data['omega'], name = 'omega')
#omk2_ILR = p.line(pd['t'], (1-(1/numpy.sqrt(2)))*235*km/data['Rilr'], name = 'omk2_ILR')
#legend_items1 = [['CR',[CR]],['ILR',[ILR]]]
#legend = Legend(items = legend_items1)
#legend.click_policy="hide"
#p.add_layout(legend, 'right')

#show(p)
save(p,'plot1.html')

In [ ]:
#for animating

pl = ps.index[(ps['r0'] == 0.65) & (ps['phi0'] == 0)]#[i for i in ps.index[(ps['r0'] == 1.95)] if x1timesc[i]!=[]] #a list of the particles to plot
idx_start = data.index[data['t'] >= Myr*1e3][0]#data.index[0.8*1.4 <= data['Rcr']/data['Rcr'][0]][0]#the index from which to start plotting
idx_end = data.index[-1]#data.index[1.2*1.4 >= data['Rcr']/data['Rcr'][0]][-1] #data.index[-1] #the index at which to stop plotting
no_points = 200 #the number of points to plot at a time; put None to plot all points up to the current index
period = 50 #the number of milliseconds between each update of the plot (the lower it is, the faster the animation)
visible_lines = l #a list of the particles the lines of which are to be made initially visible on the plot; particles not in pl have no effect
#if you want to make the lines of all particles visible except some particles s1,s2,s3 then use:
#visible_lines = [s for s in l if s in {s1,s2,s3}]

TOOLTIPS = [
    ("index", "$index"),
    ("name", "$name"),
    ("(x,y)", "($x, $y)")
]

from bokeh.io import output_notebook, show
output_notebook()

from bokeh.plotting import figure
from bokeh.models import Legend, ColumnDataSource, LabelSet, Title, Circle, HoverTool#, Label

from bokeh.palettes import Category20b
import itertools
colours = itertools.cycle(Category20b[20]) #cycle back through list when it reaches end
#if you want to plot more than 20 with different colours, then use (for up to 256):
#import all_pallettes
#colours = all_palettes['Turbo'][256][0:256-(256%np):(256//np)] #to give colours equally spaced out in that spectrum
#...
#legend_items1 = [legend_items[i:i+20] for i in range(0, len(legend_items), 20)] #to make multiple legends, as 1 legend gets truncated
#for item in legend_items1:
#    legend = Legend(items = item)

legend_items = [] #solution for legend based on: https://stackoverflow.com/questions/46730609/position-the-legend-outside-the-plot-area-with-bokeh

def modify_doc(doc):
    p = figure(title="plot", x_axis_label='x', y_axis_label='y', sizing_mode = 'stretch_both', match_aspect=True, tools = "pan,ywheel_pan,tap,box_zoom,wheel_zoom,zoom_in,zoom_out,undo,redo,reset,save", active_scroll = "wheel_zoom") #lasso_select,box_select don't seem to work with the lines
    xcols = [f'xc_{s}' for s in pl] #modify here and down below with p.line
    ycols = [f'yc_{s}' for s in pl]
    cols = xcols + ycols + ['t']
    cols1 = ['Rcr','Rilr','Rolr']
    source = ColumnDataSource(data[cols].iloc[idx_start:idx_start+2])
    source1 = ColumnDataSource(data[cols1].iloc[idx_start:idx_start+2]) #I think ColumnDataSource is mutable so don't set source1=source if you want to update them both independently
    plots = []
    for s,colour in zip(pl,colours):
        plots.append(p.line(x=f'xc_{s}', y=f'yc_{s}', line_color = colour, line_width=2, name = f'p_{s}', source=source, visible=False, selection_alpha = 0.2)) #wasn't able to mute all lines except selected, so set it to mute selected line. Use Ctrl to select multiple lines and click on no line to unselect
        if s in visible_lines:
            plots[-1].visible = True
        legend_items.append([f'particle {s}',plots])
    #p.ellipse(x=0, y=0, width='2Rcr', name = 'corotation radius', source = source1)
    p.circle(x=0, y=0, radius='Rcr', fill_color = None, line_color=Category20b[20][5], name = 'corotation circle', source=source1)
    p.circle(x=0, y=0, radius='Rilr', fill_color = None, line_color=Category20b[20][3], name = 'ILR circle', source=source1)
    p.circle(x=0, y=0, radius='Rolr', fill_color = None, line_color=Category20b[20][10], name = 'OLR circle', source=source1)
    #alternatively:
    #circ = Circle(x=0, y=0, radius='Rcr', fill_color = None, name = 'corotation circle')
    #p.add_glyph(source1, circ)
    p.add_tools(HoverTool(tooltips = TOOLTIPS, renderers = plots))
    
    #label = Label(x=70, y=70, x_units='screen', y_units='screen', text = '{} Myr'.format(source.data['t'][-1]/3.154e13))
    time = Title(text = '{} Myr'.format(source.data['t'][-1]/3.154e13), align = 'right')
    p.add_layout(time, 'above')
    
    if len(legend_items) <= 20: #only show legend if plotting 20 particles or less
        legend = Legend(items = legend_items)
        legend.click_policy="hide"
        p.add_layout(legend, 'right')

    def update():
        i = source.data['index'][-1] + 1
        new_data = data[cols].iloc[i:i+1]
        source.stream(new_data, no_points)
        new_data1 = data[cols1].iloc[i:i+1]
        source1.stream(new_data1, 1) #only want current corotation radius; no need to show previous values
        time.text = '{} Myr'.format(new_data['t'].at[i]/3.154e13)
        if i >= idx_end:
            doc.remove_periodic_callback(cb)
        
    cb = doc.add_periodic_callback(update, period)
    doc.add_root(p)

#show(modify_doc,notebook_url="http://localhost:8888")

In [ ]:
pl = [i for i in l if x1timesc[i]!=[]]#ps.index[(ps['r0'] == 1.75)]#[i for i in l if x1timesc[i]!=[]]

%matplotlib notebook
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

for s in pl:
    #for i in range(len(x1timesc[s])): #x1times[s]['from']
    #    cond = (data['t'] >= x1timesc[s][i][0]) & (data['t'] <= x1timesc[s][i][-1])
    #then indent lines up to 'plots.append(...' below into the chosen for loop
    
    #to plot the trajectory of particles when their not on x1 orbits (in which case use pl = [i for i in x1timesc if x1timesc[i]!=[]]):
    #cond = (data['t'] <= x1timesc[s][0][0]) #before it's first x1 orbit
    #for i in range(len(x1timesc[s]) - 1): #x1times[s]['from']
    #    cond = cond | ((data['t'] >= x1timesc[s][i][-1]) & (data['t'] <= x1timesc[s][i+1][0])) #between sequences of x1 orbits
    #cond = cond | (data['t'] >= x1timesc[s][-1][-1]) #after its last sequence of x1 orbits

    cond = (data['t'] >= Myr*0e3) & (data['t'] <= Myr*20e3)

    #r = numpy.sqrt((data[f'xc_{s}']**2) + (data[f'yc_{s}']**2))

    y = rats[f'rat_{s}'][rats[f'rat_{s}'].notna()][cond]
    x = data['t'][y.index][cond]/(Myr*1e3)

    #plt.plot(x, y, label=f'particle {s}', zorder=1)
    plt.scatter(x, y)
    
    #ts = x1timesc[s][0][0]
    #Lts = data[f'L_{s}'][data['t'] == ts].iat[0]
    #plt.plot(ts/(Myr*1e3), Lts, color='y')
    #circle1 = plt.Circle((ts/(Myr*1e3), Lts), color='y')
    #ax.add_patch(circle1)
    #plt.scatter(ts/(Myr*1e3), Lts, facecolor='yellow', zorder=2)
#ax.set_ylim(0,2.9e25)
#plt.plot(data['t'][cond]/(Myr*1e3),235*km*data['Rilr'][cond])
#plt.plot(data['t'][cond]/(Myr*1e3),235*km*data['Rcr'][cond])
#plt.plot(data['t'][cond]/(Myr*1e3),235*km*data['Rolr'][cond])
#plt.plot(data['t']/(Myr*1e3), ((235*km)**2)*(-0.5 + numpy.log(data['Rcr']/kpc)), color = 'y')
plt.show()
plt.savefig('plot.png', bbox_inches='tight')

In [ ]:
particles = l
index = 10000
t_end = Myr*20e3

cols = [col for s in particles for col in [f'x_{s}',f'y_{s}',f'vx_{s}',f'vy_{s}']]
cols = ['t','tend'] + cols
data1 = data.loc[[index]]
data1['tend'] = t_end
data1.to_csv("C:/init.txt", columns=cols, index=False)


In [ ]:
(3*kpc)/(2*(0.28))
#data['Rcr'].iat[10000]
1.6532142857142855e+20/kpc
#data.index[data['Rcr'] >= 1.6532142857142855e+20][0]

In [ ]:
#potential variation:

%matplotlib notebook
vc2 = (235*km)**2
Af = 0.02
b = 0.28
M = 2
Rcr0 = data['Rcr'].iat[0]
Rcr = data['Rcr'].iat[3000]
print(Rcr)

x = numpy.arange(-3*Rcr0,3*Rcr0,0.01*Rcr0)
y = numpy.arange(-3*Rcr0,3*Rcr0,0.01*Rcr0)

X, Y = numpy.meshgrid(x,y)

Z = vc2*numpy.log(numpy.sqrt((X**2)+(Y**2))/kpc) - (Af*vc2/(M*(Rcr**2)))*(((b+1)/(b+(numpy.sqrt((X**2)+(Y**2))/Rcr)))**5)*((X**2)-(Y**2))
Zlog = vc2*numpy.log(numpy.sqrt((X**2)+(Y**2))/kpc)

import matplotlib.pyplot as plt
levels = numpy.arange(-3*Rcr0,3*Rcr0,0.01*Rcr0)
fig, ax = plt.subplots()
levels = [i*2e10 for i in range(-7,7)]
CS = ax.contour(X, Y, Z, levels, colors='b')#, label='potential with perturbation')
CS.collections[-1].set_label('log with pert')
#ax.clabel(CS)
CS1 = ax.contour(X, Y, Zlog, levels, colors='y')#, label='logarithmic background')
CS1.collections[-1].set_label('log')
ax.legend()
ax.set_aspect(1)
ax.set_title('Contours of the potential (bar along x axis)')
plt.show()

R = numpy.linspace(0,3e20,10000)
log = vc2*numpy.log(R/kpc)
pert = (Af*vc2/M)*((R/Rcr)**2)*(((b+1)/(b+(R/Rcr)))**5)

potmax = log + pert
potmin = log - pert
fig, ax = plt.subplots()
plt.plot(R,potmax,label='log + pert amp')
plt.plot(R,potmin,label='log - pert amp')
ax.legend()
ax.set_title('potential against R')
ax.set_xlabel('R/m')
ax.set_ylabel('potential/$m^2s^{-2}$')
plt.show()

rat = abs(pert/log)
fig, ax = plt.subplots()
plt.plot(R,rat)
ax.set_ylim(0,1)
ax.set_title('ratio of perturbation amplitude to logarithmic background against R')
ax.set_xlabel('R/m')
ax.set_ylabel('ratio')
plt.show()

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

vc2 = (235*km)**2
Af = 0.02
b = 0.28
M = 2
omega = 80*km/kpc
Rcr0 = data['Rcr'].iat[0]

k = 2
r0 = 0.85*Rcr0
L0 = k*numpy.sqrt(vc2)*r0
E0 = 4.3*vc2*(0.5 + numpy.log(r0/kpc))

r = numpy.linspace(0.6*Rcr0, 1.2*Rcr0, 10000)
print(r)
vphi = L0/r
print(vphi)
vr = numpy.sqrt((2*E0) - (vphi**2) - (2*vc2*numpy.log(r/kpc)))
lis = [i  for i in range(vr.size) if not numpy.isnan(vr)[i]]
if len(lis) == 0:
    print('no valid values for vr')
else:
    print(r[lis[0]]/Rcr0) #the first value of r (in units of Rcr0) which yields a valid value for vr
    print(r[lis[-1]]/Rcr0) #the last value of r (in units of Rcr0) which yields a valid value for vr

rat = vr/vphi
fig, ax = plt.subplots()
plt.plot(r/Rcr0,rat,label=f'{r0/Rcr0}')
ax.legend()
#ax.set_title('potential against R')
#ax.set_xlabel('R/m')
#ax.set_ylabel('potential/$m^2s^{-2}$')
plt.show()

In [ ]:
#plotting Ej vs t:

from bokeh.plotting import figure, show, save
from bokeh.models import Legend

TOOLTIPS = [
    ("index", "$index"),
    ("name", "$name"),
    ("(x,y)", "($x, $y)")
]

p = figure(title="Ej", x_axis_label='t', y_axis_label='Ej', sizing_mode = 'stretch_both', tools = "pan,ywheel_pan,tap,box_zoom,wheel_zoom,zoom_in,zoom_out,undo,redo,reset,save", tooltips = TOOLTIPS, match_aspect=True, aspect_scale=1e-13) #use 1e-5 if plotting Ej in kgm2s-2, or 1e-13 if plotting Ej per unit mass in km2s-2; lasso_select,box_select don't seem to work with the lines
#l = ['']
#for i in range(2,20):
#    l.append('.{}'.format(i))
#print(l)

#bokeh doesn't automatically cycle through colours for different lines; see: https://stackoverflow.com/questions/39839409/when-plotting-with-bokeh-how-do-you-automatically-cycle-through-a-color-pallett, https://docs.bokeh.org/en/latest/docs/reference/palettes.html, 
from bokeh.palettes import Category20b
import itertools
colours = itertools.cycle(Category20b[20]) #cycle back through list when it reaches end
#if you want to plot more than 20 with different colours, then use (for up to 256):
#import all_pallettes
#colours = all_palettes['Turbo'][256][0:256-(256%np):(256//np)] #to give colours equally spaced out in that spectrum
#...
#legend_items1 = [legend_items[i:i+20] for i in range(0, len(legend_items), 20)] #to make multiple legends, as 1 legend gets truncated
#for item in legend_items1:
#    legend = Legend(items = item)


legend_items = [] #solution for legend based on: https://stackoverflow.com/questions/46730609/position-the-legend-outside-the-plot-area-with-bokeh

for s,colour in zip(l,colours):
    #data1 = data[data.index < min(idxs1)] #only plot up to when any particle first changes n
    #data1 = data[data.index < idxs1[s-1]] #for each particle, only plot up to when it first changes n (if it changes)
    #then change references to 'data' below to 'data1':
    x = data['t']
    y = data[f'Ej_{s}']/(1e6)
    plot = p.line(x, y, line_color = colour, line_width=2, name = f'Ej_{s}', selection_alpha = 0.2) #wasn't able to mute all lines except selected, so set it to mute selected line. Use Ctrl to select multiple lines and click on no line to unselect
    legend_items.append([f'particle {s}',[plot]])

if len(legend_items) <= 24: #only show legend if plotting 20 particles or less
    legend = Legend(items = legend_items)
    legend.click_policy="hide"
    p.add_layout(legend, 'right')

#show(p)
save(p,'plot.html')

In [ ]:
data6,data5 = data5,data6
datas,datas5 = datas5,datas
print(len(datas))
data6
#data = datas[0]

In [ ]:
data5,data = data,data5
x1times,x1times5 = x1times5,x1times
x1timesc,x1timesc5 = x1timesc5,x1timesc
apois,apois5 = apois5,apois
print(data.info(verbose=False))
print(data['t'].iat[0])

In [ ]:
data5 = None
x1times5 = None
x1timesc5 = None
apois5 = None

In [ ]:
#The equality of a particle's angular momentum values to the particle with same initial radius and angle 180 degrees opposite:

lis = {}
for s in ps.index[ps['phi0'] < 180]:
    cond = (data[f'L_{s}'] != data[f'L_{s+12}'])
    #cond = (data[f'x_{s}']**2) + (data[f'y_{s}']**2) != (data[f'x_{s+12}']**2) + (data[f'y_{s+12}']**2) #if you want to check the equality of their r values
    #cond = (data[f'vx_{s}']**2) + (data[f'vy_{s}']**2) != (data[f'vx_{s+12}']**2) + (data[f'vy_{s+12}']**2) #if you want to check the equality of their v values
    lis[s] = data.index[cond].size
print("The number of indices for which the particle's angular momentum isn't equal to that of the particle 180 degrees opposite:")
print(lis)
lis1 = sorted(lis, key = lambda x: lis[x])
lis2 = {}
for s in lis1:
    lis2[s] = lis[s]
print("That dictionary sorted from smallest to largest:")
print(lis2)
#data[['L_245','L_257']].index[data['L_245'] != data['L_257']].size
#print(data['L_245'].iat[6984] - data['L_257'].iat[6984])

# Figures for report

In [ ]:
from bokeh.io import output_notebook, show
output_notebook()

In [ ]:
#fig (4)

pl = ps.index[(ps['r0'] == 1.6) & (ps['phi0'] < 180)]#[i for i in ps.index[(ps['r0'] <= 1.25)] if x1timesc[i] != []] #a list of the particles to plot
visible_lines = l#[pl[0]] #a list of the particles the lines of which are to be made initially visible on the plot; particles not in pl have no effect
#if you want to make the lines of all particles visible except some particles s1,s2,s3 then use:
#visible_lines = [s for s in l if s not in {s1,s2,s3}]

from bokeh.plotting import figure, show, save
from bokeh.models import Legend, ColumnDataSource, HoverTool

tool = HoverTool(
    tooltips = [
        ("time", "@t"),#@t{%e}
        ("name", "$name"),
        ("(x,y)", "($x, $y)")
    ],
    formatters = {"@t": "printf"}
)

p = figure(x_axis_label = r'$$t/Gyr$$', y_axis_label = r'$$L/m^2s^{-1}$$', sizing_mode = 'stretch_both', tools = "pan,ywheel_pan,tap,box_zoom,wheel_zoom,zoom_in,zoom_out,undo,redo,reset,save")#, match_aspect=True, aspect_scale=1e0) #lasso_select,box_select don't seem to work with the lines
#p.xaxis.axis_label = r"t/Gyr"
#p.yaxis.axis_label = r"L/$$m^2s^{-1}$$"
p.add_tools(tool)

#l = ['']
#for i in range(2,20):
#    l.append('.{}'.format(i))
#print(l)

#bokeh doesn't automatically cycle through colours for different lines; see: https://stackoverflow.com/questions/39839409/when-plotting-with-bokeh-how-do-you-automatically-cycle-through-a-color-pallett, https://docs.bokeh.org/en/latest/docs/reference/palettes.html, 
from bokeh.palettes import Category20b
import itertools
colours = itertools.cycle(Category20b[20]) #cycle back through list when it reaches end
#if you want to plot more than 20 with different colours, then use (for up to 256):
#import all_pallettes
#colours = all_palettes['Turbo'][256][0:256-(256%np):(256//np)] #to give colours equally spaced out in that spectrum
#...
#legend_items1 = [legend_items[i:i+20] for i in range(0, len(legend_items), 20)] #to make multiple legends, as 1 legend gets truncated
#for item in legend_items1:
#    legend = Legend(items = item)

legend_items = [] #solution for legend based on: https://stackoverflow.com/questions/46730609/position-the-legend-outside-the-plot-area-with-bokeh

pd = {}
for s,colour in zip(pl,colours):
    plots = []
    
    #to plot the particle's x1 orbits (using either x1timesc or x1times; use 1 of the 2 for loops):
    #for i in range(len(x1timesc[s])): #x1times[s]['from']
        #cond = (data['t'] >= x1timesc[s][i][0]) & (data['t'] <= x1timesc[s][i][-1])
    #for i in x1times[s]['from']:
        #cond = (data['t'] >= x1times[s]['from'][i]) & (data['t'] <= x1times[s]['to'][i])
    #then indent lines up to 'plots.append(...' below into the chosen for loop
    
    #to plot the trajectory of particles when their not on x1 orbits (in which case use pl = [i for i in x1timesc if x1timesc[i]!=[]]):
    #cond = (data['t'] <= x1timesc[s][0][0]) #before it's first x1 orbit
    #for i in range(len(x1timesc[s]) - 1): #x1times[s]['from']
        #cond = cond | ((data['t'] >= x1timesc[s][i][-1]) & (data['t'] <= x1timesc[s][i+1][0])) #between sequences of x1 orbits
    #cond = cond | (data['t'] >= x1timesc[s][-1][-1]) #after its last sequence of x1 orbits
    
    cond = (data['t'] >= Myr*0e3) & (data['t'] <= Myr*10e3) #(0.8*ps.at[s,'r0'] <= data['Rcr']/data['Rcr'][0]) & (1.2*ps.at[s,'r0'] >= data['Rcr']/data['Rcr'][0]) #(data.index >= cr[s][0]) & (data.index <= cr[s][-1]) #data.index > -1
    
    #r = numpy.sqrt((data[f'xc_{s}']**2) + (data[f'yc_{s}']**2))
    
    #pd['x'] = data[f'xc_{s}'][cond]
    pd['y'] = data[f'L_{s}'][cond]
    pd['t'] = data['t'][cond]/(Myr*1e3)
    #pd['r'] = r[cond]*1e5
    
    source = ColumnDataSource(pd)
    
    plots.append(p.line('t', 'y', line_color = colour, line_width=2, name = f'p_{s}', visible=False, selection_alpha = 0.2, source=source)) #wasn't able to mute all lines except selected, so set it to mute selected line. Use Ctrl to select multiple lines and click on no line to unselect
    #plots.append(p.line('t', 'r', line_color = colour, line_width=2, name = f'p_{s}', visible=False, selection_alpha = 0.2, source=source)) #wasn't able to mute all lines except selected, so set it to mute selected line. Use Ctrl to select multiple lines and click on no line to unselect
    #data2 = data1[(data1[f'xc_{s}'] < data1[f'xc_{s}'].shift(1)) & (data1[f'xc_{s}'] < data1[f'xc_{s}'].shift(-1))]
    #data2 = data1.loc[data1.index.intersection(cr[s])]
    #x1 = data[f'xc_{s}'][aposr.index][cond]
    #y1 = data[f'yc_{s}'][aposr.index][cond]
    #x2 = data[f'xc_{s}'][tphi90.index][cond]
    #y2 = data[f'yc_{s}'][tphi90.index][cond]
    #plot1 = p.circle(x1, y1, color='yellow', size=7, visible=False)
    #plot2 = p.circle(x2, y2, color='blue', size=7, visible=False)
    
    #plot = p.dot(x, y, name = f'p_{s}', visible=False, selection_alpha = 0.2)
    #y2 = data[f'Ej_{s}']
    #plot1 = p.line(x, y2, line_color = colour, line_width=2, name = f'Ej_{s}', visible=False)
    
    #plots = [plot]#,plot1,plot2]
    
    if (s in visible_lines) or (f'{s}' in visible_lines):
        for plot in plots:
            plot.visible = True
    legend_items.append([f'particle {s}',plots])

if len(legend_items) <= 24: #only show legend if plotting 20 particles or less
    legend = Legend(items = legend_items)
    legend.click_policy="hide"
    p.add_layout(legend, 'right')

ILR = p.line(pd['t'], 235*km*data['Rilr'], name = 'ILR')
#OLR = p.line(pd['t'], 235*km*data['Rolr'], name = 'OLR')
CR = p.line(pd['t'], 235*km*data['Rcr'], name = 'CR')
#legend_items1 = [['CR',[CR]],['ILR',[ILR]]]
#legend = Legend(items = legend_items1)
#legend.click_policy="hide"
#p.add_layout(legend, 'right')

show(p)
save(p,'plot1.html')

In [ ]:
#fig (6)

pl = [i for i in l if x1timesc[i]!=[]]#ps.index[(ps['r0'] == 1.75)]#[i for i in l if x1timesc[i]!=[]]

%matplotlib notebook
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

for s in pl:
    #for i in range(len(x1timesc[s])): #x1times[s]['from']
    #    cond = (data['t'] >= x1timesc[s][i][0]) & (data['t'] <= x1timesc[s][i][-1])
    #then indent lines up to 'plots.append(...' below into the chosen for loop
    
    #to plot the trajectory of particles when their not on x1 orbits (in which case use pl = [i for i in x1timesc if x1timesc[i]!=[]]):
    #cond = (data['t'] <= x1timesc[s][0][0]) #before it's first x1 orbit
    #for i in range(len(x1timesc[s]) - 1): #x1times[s]['from']
    #    cond = cond | ((data['t'] >= x1timesc[s][i][-1]) & (data['t'] <= x1timesc[s][i+1][0])) #between sequences of x1 orbits
    #cond = cond | (data['t'] >= x1timesc[s][-1][-1]) #after its last sequence of x1 orbits

    cond = (data['t'] >= Myr*0e3) & (data['t'] <= Myr*20e3)

    #r = numpy.sqrt((data[f'xc_{s}']**2) + (data[f'yc_{s}']**2))

    x = data['t'][cond]/(Myr*1e3)
    y = data[f'L_{s}'][cond]

    plt.plot(x, y, label=f'particle {s}', zorder=1)
    
    ts = x1timesc[s][0][0]
    Lts = data[f'L_{s}'][data['t'] == ts].iat[0]
    #plt.plot(ts/(Myr*1e3), Lts, color='y')
    #circle1 = plt.Circle((ts/(Myr*1e3), Lts), color='y')
    #ax.add_patch(circle1)
    plt.scatter(ts/(Myr*1e3), Lts, facecolor='yellow', zorder=2)
ax.set_ylim(0,2.9e25)
plt.plot(data['t'][cond]/(Myr*1e3),235*km*data['Rilr'][cond])
plt.plot(data['t'][cond]/(Myr*1e3),235*km*data['Rcr'][cond])
#plt.plot(data['t'][cond]/(Myr*1e3),235*km*data['Rolr'][cond])
#plt.plot(data['t']/(Myr*1e3), ((235*km)**2)*(-0.5 + numpy.log(data['Rcr']/kpc)), color = 'y')
ax.set_xlabel(r'$t/Gyr$')
ax.set_ylabel(r'$L/m^2s^{-1}$')
plt.show()
plt.savefig('plot.png', bbox_inches='tight')

In [ ]:
#fig (6)

pl = [i for i in l if x1timesc[i]!=[]]#ps.index[(ps['r0'] == 1.75)]#[i for i in l if x1timesc[i]!=[]]

%matplotlib notebook
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

for s in pl:
    #for i in range(len(x1timesc[s])): #x1times[s]['from']
    #    cond = (data['t'] >= x1timesc[s][i][0]) & (data['t'] <= x1timesc[s][i][-1])
    #then indent lines up to 'plots.append(...' below into the chosen for loop
    
    #to plot the trajectory of particles when their not on x1 orbits (in which case use pl = [i for i in x1timesc if x1timesc[i]!=[]]):
    #cond = (data['t'] <= x1timesc[s][0][0]) #before it's first x1 orbit
    #for i in range(len(x1timesc[s]) - 1): #x1times[s]['from']
    #    cond = cond | ((data['t'] >= x1timesc[s][i][-1]) & (data['t'] <= x1timesc[s][i+1][0])) #between sequences of x1 orbits
    #cond = cond | (data['t'] >= x1timesc[s][-1][-1]) #after its last sequence of x1 orbits

    cond = apois[s]#(data['t'] >= Myr*0e3) & (data['t'] <= Myr*20e3)

    r = numpy.sqrt((data[f'xc_{s}']**2) + (data[f'yc_{s}']**2))/data['Rcr']

    x = data['t'][cond]/(Myr*1e3)
    y = r[cond]

    plt.scatter(x, y, label=f'particle {s}', zorder=1)
    
    tsi = data.index[data['t'] == x1timesc[s][0][0]][0]
    idx = apois[s][apois[s] >= tsi][0]
    tval = data['t'].at[idx]
    apoval = r.at[idx]
    plt.scatter(tval/(Myr*1e3), apoval, facecolor='yellow', zorder=2)
    #circle1 = plt.Circle((ts/(Myr*1e3), Lts), color='y')
    #ax.add_patch(circle1)
    #plt.scatter(ts/(Myr*1e3), Lts, facecolor='yellow', zorder=2)
#ax.set_ylim(0,1.3e20)
#plt.plot(data['t'][cond]/(Myr*1e3),data['Rilr'][cond])
#plt.plot(data['t'][cond]/(Myr*1e3),data['Rcr'][cond])
#plt.plot(data['t'][cond]/(Myr*1e3),235*km*data['Rolr'][cond])
#plt.plot(data['t']/(Myr*1e3), ((235*km)**2)*(-0.5 + numpy.log(data['Rcr']/kpc)), color = 'y')
ax.set_xlabel(r'$t/Gyr$')
ax.set_ylabel(r'$r/R_{\mathrm{CR}}$')
plt.show()
plt.savefig('plot.png', bbox_inches='tight')

In [ ]:
#fig (9): potential variation:

%matplotlib notebook
vc2 = (235*km)**2
Af = 0.02
b = 0.28
M = 2
#Rcr0 = data['Rcr'].iat[0]
Rcr = 1.6532142857142855e+20#data['Rcr'].iat[3000]
print(Rcr)

x = numpy.arange(-0.4,0.4,0.01)
y = numpy.arange(-0.4,0.4,0.01)

X, Y = numpy.meshgrid(x,y)

Z = vc2*numpy.log(numpy.sqrt((X**2)+(Y**2))*Rcr/kpc) - (Af*vc2/M)*(((b+1)/(b+(numpy.sqrt((X**2)+(Y**2)))))**5)*((X**2)-(Y**2))
Zlog = vc2*numpy.log(numpy.sqrt((X**2)+(Y**2))*Rcr/kpc)

import matplotlib.pyplot as plt
#levels = numpy.arange(-3,3,0.01)
fig, ax = plt.subplots()
levels = [i*2e10 for i in range(-3,3)]
#levels = numpy.linspace(Z.min(), Z.max(), 10)
CS = ax.contour(X, Y, Z, levels, colors='b')#, label='potential with perturbation')
CS.collections[-1].set_label('$\Phi (R, \phi, t)$')
#ax.clabel(CS)
CS1 = ax.contour(X, Y, Zlog, levels, colors='y')#, label='logarithmic background')
CS1.collections[-1].set_label('$\Phi_0(R)$')
ax.set_xlabel('$x/R_{\mathrm{CR}}$')
ax.set_ylabel('$y/R_{\mathrm{CR}}$')
ax.legend()
ax.set_aspect(1)
#ax.set_title('Contours of the potential (bar along x axis)')
plt.show()
plt.savefig('plot.png', bbox_inches='tight')

val = numpy.linspace(0,2.6e20,10000)/Rcr #note that I think it doesn't include the first point as it's log of zero, so the plot seems to actually start from 2.6e16
log = vc2*numpy.log(val*Rcr/kpc)
pert = (Af*vc2/M)*(val**2)*(((b+1)/(b+val))**5)

potmax = log + pert
potmin = log - pert
fig, ax = plt.subplots()
plt.plot(val,potmax,label='$\Phi_0 + \Phi_m$')
plt.plot(val,potmin,label='$\Phi_0 - \Phi_m$')
ax.legend()
#ax.set_title('potential against R')
ax.set_xlabel('$R/R_{\mathrm{CR}}$')
ax.set_ylabel('$\Phi$')
plt.show()

'''
rat = abs(pert/log)
fig, ax = plt.subplots()
plt.plot(R,rat)
ax.set_ylim(0,1)
ax.set_title('ratio of perturbation amplitude to logarithmic background against R')
ax.set_xlabel('R/m')
ax.set_ylabel('ratio')
plt.show()
'''

In [ ]:
#fig(15)

pl = [i for i in l if x1timesc[i] != []] #a list of the particles to plot
visible_lines = l#[pl[0]] #a list of the particles the lines of which are to be made initially visible on the plot; particles not in pl have no effect
#if you want to make the lines of all particles visible except some particles s1,s2,s3 then use:
#visible_lines = [s for s in l if s not in {s1,s2,s3}]

from bokeh.plotting import figure, show, save
from bokeh.models import Legend, ColumnDataSource, HoverTool, Span, BoxAnnotation

tool = HoverTool(
    tooltips = [
        ("time", "@t"),#@t{%e}
        ("name", "$name"),
        ("(x,y)", "($x{%e}, $y{%e})")
    ],
    formatters = {"@t": "printf",
                 "$x": "printf",
                 "$y": "printf"}
)

p = figure(x_axis_label=r'$$t/Gyr$$', y_axis_label=r'$$\frac{r_{\mathrm{min}}}{r_{\mathrm{max}}}$$', x_range = (0,10), sizing_mode = 'stretch_both', tools = "pan,ywheel_pan,tap,box_zoom,wheel_zoom,zoom_in,zoom_out,undo,redo,reset,save")#, match_aspect=True, aspect_scale=1e0) #lasso_select,box_select don't seem to work with the lines
# y_range=(rats.min().min(), rats.max().max()),
#p.yaxis.ticker = numpy.linspace(0,1,11)
p.yaxis.ticker.desired_num_ticks = 11
#p = figure(title="other", x_axis_label='x axis', y_axis_label='y axis', sizing_mode = 'stretch_both', tools = "pan,ywheel_pan,tap,box_zoom,wheel_zoom,zoom_in,zoom_out,undo,redo,reset,save")#, match_aspect=True, aspect_scale=1e0) #lasso_select,box_select don't seem to work with the lines
p.add_tools(tool)

#l = ['']
#for i in range(2,20):
#    l.append('.{}'.format(i))
#print(l)

#bokeh doesn't automatically cycle through colours for different lines; see: https://stackoverflow.com/questions/39839409/when-plotting-with-bokeh-how-do-you-automatically-cycle-through-a-color-pallett, https://docs.bokeh.org/en/latest/docs/reference/palettes.html, 
from bokeh.palettes import Category20b
import itertools
colours = itertools.cycle(Category20b[20])#cycle back through list when it reaches end
#if you want to plot more than 20 with different colours, then use (for up to 256):
#import all_pallettes
#colours = all_palettes['Turbo'][256][0:256-(256%np):(256//np)] #to give colours equally spaced out in that spectrum
#...
#legend_items1 = [legend_items[i:i+20] for i in range(0, len(legend_items), 20)] #to make multiple legends, as 1 legend gets truncated
#for item in legend_items1:
#    legend = Legend(items = item)

#from bokeh.palettes import all_palettes
#skip = 17
#cs = all_palettes['Turbo'][256][0:skip*13:skip] #to give colours equally spaced out in that spectrum

legend_items = [] #solution for legend based on: https://stackoverflow.com/questions/46730609/position-the-legend-outside-the-plot-area-with-bokeh

pd = {}
#legplots = {}
#for key in vr0:
#    legplots[abs(key)] = []
'''
colours = {}
i = 0
for key in legplots:
    colours[key] = cs[i]#Category20b[15][2:][i]
    i += 1
for s in pl:
'''
for s,colour in zip(pl,colours):
    #vr0s = abs(ps['vr0'].at[s])
    plots = []
    
    #to plot the particle's x1 orbits (using either x1timesc or x1times; use 1 of the 2 for loops):
    #for i in range(len(x1timesc[s])): #x1times[s]['from']
        #cond = (data['t'] >= x1timesc[s][i][0]) & (data['t'] <= x1timesc[s][i][-1])
    #for i in x1times[s]['from']:
        #cond = (data['t'] >= x1times[s]['from'][i]) & (data['t'] <= x1times[s]['to'][i])
    #then indent lines up to 'plots.append(...' below into the chosen for loop
    
    #to plot the trajectory of particles when their not on x1 orbits (in which case use pl = [i for i in x1timesc if x1timesc[i]!=[]]):
    #cond = (data['t'] <= x1timesc[s][0][0]) #before it's first x1 orbit
    #for i in range(len(x1timesc[s]) - 1): #x1times[s]['from']
        #cond = cond | ((data['t'] >= x1timesc[s][i][-1]) & (data['t'] <= x1timesc[s][i+1][0])) #between sequences of x1 orbits
    #cond = cond | (data['t'] >= x1timesc[s][-1][-1]) #after its last sequence of x1 orbits
    
    cond = (data['t'] >= Myr*0e3) & (data['t'] <= Myr*20e3) #(0.8*ps.at[s,'r0'] <= data['Rcr']/data['Rcr'][0]) & (1.2*ps.at[s,'r0'] >= data['Rcr']/data['Rcr'][0]) #(data.index >= cr[s][0]) & (data.index <= cr[s][-1]) #data.index > -1
    
    #r = numpy.sqrt((data[f'xc_{s}']**2) + (data[f'yc_{s}']**2))
    #for plotting the azimuths of apoapses over time:
    #phic = abs(numpy.arctan2(data[f'yc_{s}'], data[f'xc_{s}']))[apois[s]]/numpy.pi
    
    #vr = (data[f'vy_{s}']*data[f'y_{s}'] + data[f'vx_{s}']*data[f'x_{s}'])/r
    #vphi = (data[f'vy_{s}']*data[f'x_{s}'] - data[f'vx_{s}']*data[f'y_{s}'])/r
    
    #pd['x'] = data[f'xc_{s}'][cond]
    pd['y'] = rats[f'rat_{s}'][rats[f'rat_{s}'].notna()][cond]#[maxyis[s]] #/data['Rcr'].iat[0]
    pd['t'] = (data['t'][pd['y'].index][cond]/(Myr*1e3))#[apois[s]]
    #pd['r'] = r[cond]*1e5
    
    source = ColumnDataSource(pd)
    
    plots.append(p.circle('t', 'y', color = colour, name = f'p_{s}', visible=False, selection_alpha = 0.2, source=source)) #wasn't able to mute all lines except selected, so set it to mute selected line. Use Ctrl to select multiple lines and click on no line to unselect
    #plots.append(p.line(data['t'][cond]/(Myr*1e3), (data[f'L_{s}']/data[f'L_{s}'].iat[0])[cond], line_color = 'red', name = f'p_{s}', visible=False, selection_alpha = 0.2)) #wasn't able to mute all lines except selected, so set it to mute selected line. Use Ctrl to select multiple lines and click on no line to unselect
    #plots.append(p.line('t', 'y', line_color = colour, line_width=2, name = f'p_{s}', visible=False, selection_alpha = 0.2, source=source))
    #plots.append(p.line('t', 'r', line_color = colour, line_width=2, name = f'p_{s}', visible=False, selection_alpha = 0.2, source=source))
    #plots.append(p.line(Lmeans['t']/(Myr*1e3), Lmeans[f'L_{s}'], line_color = colours[vr0s], line_width=2, name = f'p_{s}', visible=False, selection_alpha = 0.2))
    #plots.append(p.circle(vr0s, drops[s], color = colours[vr0s], name = f'p_{s}', visible=False, selection_alpha = 0.2))
    #ts = [el/(Myr*1e3) for lis in x1timesc[s] for el in lis]
    #for t in ts:
    #    span = Span(location = t, dimension = 'height') #drawing a vertical line on the plot where the 90 degree azimuth occurs when the particle is on an x1 orbit
    #    p.add_layout(span)
    
    #for ts in x1timesc[s]:
    #    box = BoxAnnotation(left = ts[0]/(Myr*1e3), right = ts[-1]/(Myr*1e3), fill_alpha = 0.2, fill_color = colour) #shadin the region when it's on an x1 orbit
    #    p.add_layout(box)
    
    #data2 = data1[(data1[f'xc_{s}'] < data1[f'xc_{s}'].shift(1)) & (data1[f'xc_{s}'] < data1[f'xc_{s}'].shift(-1))]
    #data2 = data1.loc[data1.index.intersection(cr[s])]
    #x1 = data[f'xc_{s}'][aposr.index][cond]
    #y1 = data[f'yc_{s}'][aposr.index][cond]
    #x2 = data[f'xc_{s}'][tphi90.index][cond]
    #y2 = data[f'yc_{s}'][tphi90.index][cond]
    #plot1 = p.circle(x1, y1, color='yellow', size=7, visible=False)
    #plot2 = p.circle(x2, y2, color='blue', size=7, visible=False)
    
    #plot = p.dot(x, y, name = f'p_{s}', visible=False, selection_alpha = 0.2)
    #y2 = data[f'Ej_{s}']
    #plot1 = p.line(x, y2, line_color = colour, line_width=2, name = f'Ej_{s}', visible=False)
    
    #plots = [plot]#,plot1,plot2]
    
    if (s in visible_lines) or (f'{s}' in visible_lines):
        for plot in plots:
            plot.visible = True
    legend_items.append([f'particle {s}',plots])
    #legplots[vr0s] += plots
    
#legend_items = [[f'{key}',val] for key,val in legplots.items() if val != []]
if len(legend_items) <= 25: #only show legend if plotting 20 particles or less
    legend = Legend(items = legend_items)
    legend.click_policy="hide"
    p.add_layout(legend, 'right')

#ILR = p.line(pd['t'], 235*km*data['Rilr'], name = 'ILR')
#ILR = p.line(data['t']/(Myr*1e3), 235*km*data['Rilr'], name = 'ILR')
#OLR = p.line(pd['t'], 235*km*data['Rolr'], name = 'OLR')
#CR = p.line(pd['t'], 235*km*data['Rcr'], name = 'CR')
#CR = p.line(data['t']/(Myr*1e3), 235*km*data['Rcr'], name = 'CR')
#p.line(data['t']/(Myr*1e3), ((235*km)**2)*(-0.5 + numpy.log(data['Rcr']/kpc)))
#legend_items1 = [['CR',[CR]],['ILR',[ILR]]]
#legend = Legend(items = legend_items1)
#legend.click_policy="hide"
#p.add_layout(legend, 'right')

show(p)
#save(p,'plot3.html')

In [ ]:
#fig(2)

%matplotlib notebook
import matplotlib.pyplot as plt

fig, (ax1,ax2) = plt.subplots(1, 2)#, subplot_kw=dict(projection='polar'))

s = ps.index[(ps['r0'] == 1.05) & (ps['phi0'] == 0)][0]
cond = (data['t'] >= Myr*4e3) & (data['t'] <= Myr*10e3)
y = data[f'yc_{s}'][cond]/kpc
x = data[f'xc_{s}'][cond]/kpc
phic = numpy.arctan2(data[f'yc_{s}'], data[f'xc_{s}'])[cond]
r = numpy.sqrt((data[f'xc_{s}']**2) + (data[f'yc_{s}']**2))[cond]/kpc
ax1.plot(x, y, label=f'particle {s}', zorder=2)
lim = 9
ax1.set_ylim(-lim,lim)
ax1.set_xlim(-lim,lim)
ax1.grid(True, zorder=1)
ax1.set_aspect('equal')
ax1.set_xlabel(r'$x/kpc$')
ax1.set_ylabel(r'$y/kpc$')

s = ps.index[(ps['r0'] == 1.75) & (ps['phi0'] == 0)][0]
cond = (data['t'] >= Myr*4e3) & (data['t'] <= Myr*10e3)
y = data[f'yc_{s}'][cond]/kpc
x = data[f'xc_{s}'][cond]/kpc
phic = numpy.arctan2(data[f'yc_{s}'], data[f'xc_{s}'])[cond]
r = numpy.sqrt((data[f'xc_{s}']**2) + (data[f'yc_{s}']**2))[cond]/kpc
ax2.plot(x, y, label=f'particle {s}', zorder=2)
lim = 21
ax2.set_ylim(-lim,lim)
ax2.set_xlim(-lim,lim)
ax2.grid(True, zorder=1)
ax2.set_aspect('equal')
ax2.set_xlabel(r'$x/kpc$')
ax2.set_ylabel(r'$y/kpc$')

fig.tight_layout()

plt.show()
plt.savefig('plot.png', bbox_inches='tight')

In [ ]:
#fig(16)

%matplotlib notebook
import matplotlib.pyplot as plt

fig, (ax1,ax2) = plt.subplots(1, 2, subplot_kw=dict(projection='polar'))

s = ps.index[(ps['r0'] == 1.05) & (ps['phi0'] == 0)][0]
cond = (data['t'] >= Myr*4e3) & (data['t'] <= Myr*10e3)
y = data[f'yc_{s}'][cond]/kpc
x = data[f'xc_{s}'][cond]/kpc
phic = numpy.arctan2(data[f'yc_{s}'], data[f'xc_{s}'])[cond]
r = (numpy.sqrt((data[f'xc_{s}']**2) + (data[f'yc_{s}']**2))/data['Rcr'])[cond]
ax1.plot(phic, r, label=f'particle {s}', zorder=1)
lim = 9
#ax1.set_ylim(-lim,lim)
#ax1.set_xlim(-lim,lim)
ax1.grid(True, zorder=2)
#ax1.set_aspect('equal')
#ax1.set_xlabel(r'$x/kpc$')
#ax1.set_ylabel(r'$y/kpc$')

s = ps.index[(ps['r0'] == 1.75) & (ps['phi0'] == 0)][0]
cond = (data['t'] >= Myr*4e3) & (data['t'] <= Myr*10e3)
y = data[f'yc_{s}'][cond]/kpc
x = data[f'xc_{s}'][cond]/kpc
phic = numpy.arctan2(data[f'yc_{s}'], data[f'xc_{s}'])[cond]
r = (numpy.sqrt((data[f'xc_{s}']**2) + (data[f'yc_{s}']**2))/data['Rcr'])[cond]
ax2.plot(phic, r, label=f'particle {s}', zorder=1)
lim = 21
#ax2.set_ylim(-lim,lim)
#ax2.set_xlim(-lim,lim)
ax2.grid(True, zorder=2)
#ax2.set_aspect('equal')
#ax2.set_xlabel(r'$x/kpc$')
#ax2.set_ylabel(r'$y/kpc$')

fig.tight_layout()

plt.show()
plt.savefig('plot.png', bbox_inches='tight')

In [ ]:
#L vs t for

pl = ps.index[((ps['r0'] == 1.5) & (ps['phi0'] == 0)) | (ps['r0'] == 1.75) & (ps['phi0'] == 90)]

%matplotlib notebook
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

for s in pl:
    #for i in range(len(x1timesc[s])): #x1times[s]['from']
    #    cond = (data['t'] >= x1timesc[s][i][0]) & (data['t'] <= x1timesc[s][i][-1])
    #then indent lines up to 'plots.append(...' below into the chosen for loop
    
    #to plot the trajectory of particles when their not on x1 orbits (in which case use pl = [i for i in x1timesc if x1timesc[i]!=[]]):
    #cond = (data['t'] <= x1timesc[s][0][0]) #before it's first x1 orbit
    #for i in range(len(x1timesc[s]) - 1): #x1times[s]['from']
    #    cond = cond | ((data['t'] >= x1timesc[s][i][-1]) & (data['t'] <= x1timesc[s][i+1][0])) #between sequences of x1 orbits
    #cond = cond | (data['t'] >= x1timesc[s][-1][-1]) #after its last sequence of x1 orbits

    cond = (data['t'] >= Myr*0e3) & (data['t'] <= Myr*20e3)

    #r = numpy.sqrt((data[f'xc_{s}']**2) + (data[f'yc_{s}']**2))

    x = data['t'][cond]/(Myr*1e3)
    y = data[f'L_{s}'][cond]

    plt.plot(x, y, label=f'particle {s}', zorder=1)
    
    #ts = x1timesc[s][0][0]
    #Lts = data[f'L_{s}'][data['t'] == ts].iat[0]
    #plt.plot(ts/(Myr*1e3), Lts, color='y')
    #circle1 = plt.Circle((ts/(Myr*1e3), Lts), color='y')
    #ax.add_patch(circle1)
    #plt.scatter(ts/(Myr*1e3), Lts, facecolor='yellow', zorder=2)
#ax.set_ylim(0,2.9e25)
plt.plot(data['t'][cond]/(Myr*1e3),235*km*data['Rolr'][cond])
plt.plot(data['t'][cond]/(Myr*1e3),235*km*data['Rcr'][cond])
#plt.plot(data['t'][cond]/(Myr*1e3),235*km*data['Rolr'][cond])
#plt.plot(data['t']/(Myr*1e3), ((235*km)**2)*(-0.5 + numpy.log(data['Rcr']/kpc)), color = 'y')
ax.set_xlabel(r'$t/Gyr$')
ax.set_ylabel(r'$L/m^2s^{-1}$')
plt.show()
plt.savefig('plot.png', bbox_inches='tight')

In [ ]:
#attempt with holoviews; didn't get it working:
#for plotting lots of data: http://holoviews.org/user_guide/Large_Data.html#element-types-supported-for-datashading
#only got it working within notebook in env11; I suspect there's another package interfering
#even then wasn't very useful when trying to plot lots of data; zooming in on rasterised plot was slow and didn't show the points at higher resolution, just zoomed in on pixels
#didn't get 'stretch_both' working properly though at all
#note that some things here might interfere with the animation code cell and produce an error when you try to run it

pl = [281,282]#[i for i in x1timesc if x1timesc[i]!=[]]# if i in ps.index[(ps['r0'] >= 1.15)]]# & (ps['phi0'] == 0)] #a list of the particles to plot
visible_lines = l #a list of the particles the lines of which are to be made initially visible on the plot; particles not in pl have no effect
#if you want to make the lines of all particles visible except some particles s1,s2,s3 then use:
#visible_lines = [s for s in l if s not in {s1,s2,s3}]

import datashader as ds
import holoviews as hv
from holoviews import opts
from holoviews.operation.datashader import datashade, shade, dynspread, spread, rasterize
from holoviews.operation import decimate
hv.extension('bokeh')

#from bokeh.plotting import figure, show, save
#from bokeh.models import Legend, ColumnDataSource, HoverTool
'''
tool = HoverTool(
    tooltips = [
        ("time", "@t"),#@t{%e}
        ("name", "$name"),
        ("(x,y)", "($x, $y)")
    ],
    formatters = {"@t": "printf"}
)

p = figure(title="other", x_axis_label='x axis', y_axis_label='y axis', sizing_mode = 'stretch_both', tools = "pan,ywheel_pan,tap,box_zoom,wheel_zoom,zoom_in,zoom_out,undo,redo,reset,save")#, match_aspect=True, aspect_scale=1e0) #lasso_select,box_select don't seem to work with the lines
p.add_tools(tool)

#l = ['']
#for i in range(2,20):
#    l.append('.{}'.format(i))
#print(l)
'''
#bokeh doesn't automatically cycle through colours for different lines; see: https://stackoverflow.com/questions/39839409/when-plotting-with-bokeh-how-do-you-automatically-cycle-through-a-color-pallett, https://docs.bokeh.org/en/latest/docs/reference/palettes.html, 
from bokeh.palettes import Category20b
import itertools
colours = itertools.cycle(Category20b[20]) #cycle back through list when it reaches end
#if you want to plot more than 20 with different colours, then use (for up to 256):
#import all_pallettes
#colours = all_palettes['Turbo'][256][0:256-(256%np):(256//np)] #to give colours equally spaced out in that spectrum
#...
#legend_items1 = [legend_items[i:i+20] for i in range(0, len(legend_items), 20)] #to make multiple legends, as 1 legend gets truncated
#for item in legend_items1:
#    legend = Legend(items = item)

legend_items = [] #solution for legend based on: https://stackoverflow.com/questions/46730609/position-the-legend-outside-the-plot-area-with-bokeh

pd = {}
plots = []
for s,colour in zip(pl,colours):
    #plots = []
    
    #to plot the particle's x1 orbits (using either x1timesc or x1times; use 1 of the 2 for loops):
    #for i in range(len(x1timesc[s])): #x1times[s]['from']
        #cond = (data['t'] >= x1timesc[s][i][0]) & (data['t'] <= x1timesc[s][i][-1])
    #for i in x1times[s]['from']:
        #cond = (data['t'] >= x1times[s]['from'][i]) & (data['t'] <= x1times[s]['to'][i])
    #then indent lines up to 'plots.append(...' below into the chosen for loop
    
    #to plot the trajectory of particles when their not on x1 orbits (in which case use pl = [i for i in x1timesc if x1timesc[i]!=[]]):
    #cond = (data['t'] <= x1timesc[s][0][0]) #before it's first x1 orbit
    #for i in range(len(x1timesc[s]) - 1): #x1times[s]['from']
        #cond = cond | ((data['t'] >= x1timesc[s][i][-1]) & (data['t'] <= x1timesc[s][i+1][0])) #between sequences of x1 orbits
    #cond = cond | (data['t'] >= x1timesc[s][-1][-1]) #after its last sequence of x1 orbits
    
    cond = (data['t'] >= 0) & (data['t'] <= Myr*10e3) #(0.8*ps.at[s,'r0'] <= data['Rcr']/data['Rcr'][0]) & (1.2*ps.at[s,'r0'] >= data['Rcr']/data['Rcr'][0]) #(data.index >= cr[s][0]) & (data.index <= cr[s][-1]) #data.index > -1
    
    #r = numpy.sqrt((data[f'xc_{s}']**2) + (data[f'yc_{s}']**2))
    
    pd['x'] = data[f'xc_{s}'][cond]
    pd['y'] = data[f'L_{s}'][cond]
    pd['t'] = data['t'][cond]/(Myr*1e3)
    
    #source = ColumnDataSource(pd)
    #hv.Path(source.data,'t','y')
    plots.append(hv.Path(pd, ['t','y']).opts(color=colour))#, datatype=['dictionary']) #https://holoviews.org/user_guide/Annotating_Data.html
    #plots.append(p.line('t', 'y', line_color = colour, line_width=2, name = f'p_{s}', visible=False, selection_alpha = 0.2, source=source)) #wasn't able to mute all lines except selected, so set it to mute selected line. Use Ctrl to select multiple lines and click on no line to unselect
    
    #data2 = data1[(data1[f'xc_{s}'] < data1[f'xc_{s}'].shift(1)) & (data1[f'xc_{s}'] < data1[f'xc_{s}'].shift(-1))]
    #data2 = data1.loc[data1.index.intersection(cr[s])]
    #x1 = data[f'xc_{s}'][aposr.index][cond]
    #y1 = data[f'yc_{s}'][aposr.index][cond]
    #x2 = data[f'xc_{s}'][tphi90.index][cond]
    #y2 = data[f'yc_{s}'][tphi90.index][cond]
    #plot1 = p.circle(x1, y1, color='yellow', size=7, visible=False)
    #plot2 = p.circle(x2, y2, color='blue', size=7, visible=False)
    
    #plot = p.dot(x, y, name = f'p_{s}', visible=False, selection_alpha = 0.2)
    #y2 = data[f'Ej_{s}']
    #plot1 = p.line(x, y2, line_color = colour, line_width=2, name = f'Ej_{s}', visible=False)
    
    #plots = [plot]#,plot1,plot2]
    '''
    if (s in visible_lines) or (f'{s}' in visible_lines):
        for plot in plots:
            plot.visible = True
    legend_items.append([f'particle {s}',plots])

if len(legend_items) <= 20: #only show legend if plotting 20 particles or less
    legend = Legend(items = legend_items)
    legend.click_policy="hide"
    p.add_layout(legend, 'right')

#show(p)
save(p,'plot1.html')
'''
#show(hv.render(plot)) #sort of based on: https://discourse.holoviz.org/t/attributeerror-document-object-has-no-attribute-hold/3272/5; see: https://holoviews.org/user_guide/Plots_and_Renderers.html; update: upgraded panel and issue fixed
overlay = hv.Overlay(plots)#.opts(responsive=True) #http://holoviews.org/user_guide/Composing_Elements.html#overlay
#hv.save(overlay,'plot4.html') #https://holoviews.org/user_guide/Plots_and_Renderers.html#saving-and-rendering
rasterize(overlay)

In [ ]:
#testing to do with condition for x1 orbits

a1 = numpy.array([1,4,6,9,13,17,24,29,35,38])
a2 = a1[1:]
a3 = numpy.array([2,3,7,10,11,12,15,16,25,31,32,36])
#print(a2)
a4 = []
for i in range(len(a1[:-1])):
    n = numpy.arange(a1[i]+1,a2[i])
    #print(n)
    n1 = numpy.intersect1d(n,a3)
    #print(n1.size)
    if n1.size == 2:
        a4.append(a1[i])
print(a4)
#[1,13,29]

as1 = pandas.Series(a1)
as2 = pandas.Series(a3)

as3 = as1[as1%3==0]
print(as3.index)
print(as3)
as4 = as3[as3.index.to_series() - as3.index.to_series().shift(1,fill_value=-2) >= 2]

print(as4.index)
print(as4)
#as1[numpy.intersect1d(numpy.arange(as1.shift(1),as1),as3) == 2]

In [ ]:
#old condition for x1 orbits

apoapses = {}

for s in l:
    sub = data[(data[f'r_{s}'] > data[f'r_{s}'].shift(1)) & (data[f'r_{s}'] > data[f'r_{s}'].shift(-1))]
    cond1 = (abs(sub[f'phic_{s}']) < (40/180)*numpy.pi)
    cond2 = (abs(sub[f'phic_{s}']) > (320/180)*numpy.pi)
    cond3 = (abs(sub[f'phic_{s}']) < (140/180)*numpy.pi) & (abs(sub[f'phic_{s}']) < (220/180)*numpy.pi)
    sub1 = sub[cond1 | cond2 | cond3]
    print(sub.info())
    apoapses[s] = sub1.index

print(apoapses[3])
#data.loc[apoapses[3]]

In [ ]:
#testing:

dic = {'a': [765,25,-6543,3459,872,4,-98765,1236543,234,-876,36,875],
      'b': [987,34,23,541,23,765,6,8762,347,643,27,63]
      }

a = pandas.DataFrame.from_dict(dic)
(a['a'] > a['a'].shift(1)) & (a['a'] > a['a'].shift(-1))
for v in a['a']:
    print('v is',v)
sub = a[(a['a'] > 0) & (a['a'].shift(1) < 0)]
print(sub.index)
x = []
for i in sub.index:
    print([a['a'].loc[i-1],a['a'].loc[i]])
    x.append(numpy.interp(0,[a['a'].loc[i-1],a['a'].loc[i]],[a['b'].loc[i-1],a['b'].loc[i]]))
print(x)

In [ ]:
#for testing various possible forms of adjust(...):

"""    
    #phi = df.apply(lambda x: x.min(),1)
    #phi = df.apply(lambda x: abs(x-phi2),1)
    #df.idxmin(1).dropna()

    #mindifs = df.min(1)
    
    #df1[df1 - phi2 == mindifs]

    df = pandas.DataFrame({'minus2pi': abs(phi1-(2*numpy.pi)-phi2),
                          'nothing': abs(phi1-phi2),
                          'plus2pi': abs(phi1+(2*numpy.pi)-phi2)})

    '''
    phis = [phi1-(2*numpy.pi),phi1,phi1+(2*numpy.pi)]
    diffs = [abs(p-phi2) for p in phis]
    phi = phis[diffs.index(min(diffs))]
    return (phi > phi2)
    '''

    def absdif(f):
        return abs(f-s)
    
    #a = df1.apply(absdif,1)


    df = pandas.DataFrame({'minus2pi': phi1-(2*numpy.pi),
                        'nothing': phi1,
                        'plus2pi': phi1+(2*numpy.pi)})

"""

def gt1(phi1,phi2):
    s = phi2
    
    df = pandas.DataFrame({'minus2pi': phi1-(2*numpy.pi),
                        'nothing': phi1,
                        'plus2pi': phi1+(2*numpy.pi)})
    
    df2 = df.apply(lambda x: abs(x-s))
    s2 = df2.idxmin(1)
    l1 = []
    for idx,row in df.iterrows():
        l1.append(row[s2[idx]])
    return pandas.Series(l1,index=s.index)

def gt2(phi1,phi2):
    s = phi2
    
    df = pandas.DataFrame({'minus2pi': phi1-(2*numpy.pi),
                        'nothing': phi1,
                        'plus2pi': phi1+(2*numpy.pi)})
    
    l1 = []
    for idx,row in df.iterrows():
        l1.append(min(row,key = lambda x: abs(x-s[idx])))
        #print(min(row,key = lambda x: abs(x-s[idx])))
    return pandas.Series(l1,index=s.index)

def gt3(phi1,phi2):
    s = phi2
    
    df = pandas.DataFrame({'minus2pi': phi1-(2*numpy.pi),
                        'nothing': phi1,
                        'plus2pi': phi1+(2*numpy.pi)})
    
    idx = numpy.argmin(abs(df.sub(s, axis=0)).values, axis=1)
    
    return pandas.Series(numpy.choose(idx, df.values.T), index=s.index)

def gt4(phi1,phi2):
    s = phi2
    
    df = pandas.DataFrame({'minus2pi': phi1-(2*numpy.pi),
                        'nothing': phi1,
                        'plus2pi': phi1+(2*numpy.pi)})
    
    r = df.sub(s, axis=0).abs().idxmin(axis=1)    
    
    return pandas.Series(df.unstack().loc[zip(r.values, r.index)].values,index=s.index)

def gt5(phi1,phi2):
    s = phi2
    
    df = pandas.DataFrame({'minus2pi': phi1-(2*numpy.pi),
                        'nothing': phi1,
                        'plus2pi': phi1+(2*numpy.pi)})
    
    idx, cols = df.apply(lambda col: (col-s).abs()).idxmin(axis=1).factorize()
    
    return df.reindex(cols, axis=1).to_numpy()[numpy.arange(len(df)), idx]

phic = numpy.arctan2(data[f'yc_{2}'], data[f'xc_{2}'])
funcs = [gt3]
for func in funcs:
    t1 = time.time()
    display(func(phic,phic.shift(3)))
    t2 = time.time()
    print(t2-t1)


In [ ]:
#testing reindex:

s1 = pandas.Series(list(range(11,26)),index=[j for i in range(5) for j in range((6*i)+1,(6*i)+4)])
s2 = pandas.Series(list(range(31,61,2)),index=[j for i in range(5) for j in range((6*i)+4,(6*i)+7)])

#display(s1)
display(s2)

'''
for i in s1.index:
    a = s2.index[s2.index>i][0]
    #print(a)
l1 = [s2.index[s2.index>i][0] for i in s1.index]
s1n = s1.reindex(l1)
'''

s2n = s2.reindex(s1.index,method='bfill')
display(s1)
display(s2n)

display(s2n-s1)

'''
[20,19,18,
23,22,21,
26,25,24,
29,28,27,
32,31,30]
'''

In [ ]:
#old condition for CR orbits; simple but has issues with e.g. comparing 170 degrees and -170 degrees
cr = {}

for s in l[200:220]:
    phic = numpy.arctan2(data[f'yc_{s}'], data[f'xc_{s}'])
    cond1 = (phic < phic.shift(3)) & (phic < phic.shift(-3))
    cond2 = (phic > phic.shift(3)) & (phic > phic.shift(-3))
    sub = data[cond1 | cond2]
    sub1 = sub[abs(phic[sub.index] - phic[sub.index].shift(1))%(2*numpy.pi) < numpy.pi]
    print(sub1.info())
    cr[s] = sub1.index

#print(cr[24])
#data.loc[apoapses[3]]

In [ ]:
data[data==3.154e12]

In [ ]:
cols = [f'n_{s}' for s in l]
datasub = data[cols]
datasub[datasub['n_1'] != 1]
print([datasub[f'n_{s}'].min() for s in l])

In [ ]:
import numpy

for s in l:
    data[f'v_{s}'] = numpy.sqrt((data[f'vx_{s}']**2) + (data[f'vy_{s}']**2))
    data[f'a_{s}'] = numpy.sqrt((data[f'ax_{s}']**2) + (data[f'ay_{s}']**2))
    data[f'KE_{s}'] = 0.5*(data[f'v_{s}']**2)
    data[f'r_{s}'] = numpy.sqrt((data[f'x_{s}']**2) + (data[f'y_{s}']**2))
    data[f'phi_{s}'] = numpy.arctan2(data[f'y_{s}'], data[f'x_{s}'])
    data[f'phic_{s}'] = data[f'phi_{s}'] - data['phib']
    data[f'xc_{s}'] = data[f'r_{s}']*numpy.cos(data[f'phic_{s}'])
    data[f'yc_{s}'] = data[f'r_{s}']*numpy.sin(data[f'phic_{s}'])
    data[f'vr_{s}'] = (1/data[f'r_{s}'])*((data[f'vy_{s}']*data[f'y_{s}']) + (data[f'vx_{s}']*data[f'x_{s}']))
    data[f'vphi_{s}'] = (1/data[f'r_{s}'])*((data[f'vy_{s}']*data[f'x_{s}']) - (data[f'vx_{s}']*data[f'y_{s}']))
    data[f'phidot_{s}'] = data[f'vphi_{s}']/data[f'r_{s}']
    data[f'phidotc_{s}'] = data[f'phidot_{s}'] - data['omega']
    data[f'vxc_{s}'] = data[f'vr_{s}']*numpy.cos(data[f'phic_{s}']) - data[f'r_{s}']*data[f'phidotc_{s}']*numpy.sin(data[f'phic_{s}'])
    data[f'vyc_{s}'] = data[f'vr_{s}']*numpy.sin(data[f'phic_{s}']) + data[f'r_{s}']*data[f'phidotc_{s}']*numpy.cos(data[f'phic_{s}'])
    data[f'L_{s}'] = data[f'r_{s}']*data[f'vphi_{s}']
    data[f'E_{s}'] = data[f'KE_{s}'] + data[f'PE_{s}']
    data[f'Ej_{s}'] = data[f'E_{s}'] - (data['omega']*data[f'L_{s}'])

data['2Rcr'] = 2*data['Rcr']

In [ ]:
data

In [ ]:
#find which particles have their n changed, and the index when n of any particle first changes to non-zero value:

idxs1 = []
for s in l:
    sub = data[data[f'n_{s}'] != 0][f'n_{s}']
    if sub.size != 0:
        idx1 = sub.index[0]
    else:
        idx1 = data.index[-1]
    idxs1.append(idx1)
print(min(idxs1))
print(idxs1)

#list of particles which don't have their n changed from zero:
nochangen = [i+1 for i in range(len(idxs1)) if idxs1[i] == data.index[-1]]
#list of particles which do have their n changed from zero:
changen = [i+1 for i in range(len(idxs1)) if idxs1[i] != data.index[-1]]
print(nochangen)
print(changen)

In [ ]:
#assuming Ej is completely constant up to a point:

print('particle, constant Ej, index of row of first deviation, time at first deviation')
for s in l:
    cval = data.at[0,f'Ej_{s}']
    if data.index[data[f'Ej_{s}'] != cval].size == 0:
        print(f'Ej is constant for particle {s}')
    else:
        #data[data['Ej_1'] != cval]['Ej_1'].iat[0]
        idx = data.index[data[f'Ej_{s}'] != cval][0]
        tim = data.at[idx,'t']
        print(s, cval, idx, tim, sep=',')

In [ ]:
for s in l:
    cval = data.at[0,f'Ej_{s}']
    shifted = data[f'Ej_{s}'].shift(1)
    shifted.at[0] = cval
    diff = data[f'Ej_{s}'] - shifted #each value minus the previous value
    print(f'Instances when Ej of particle {s} decreases after first deviation:')
    print(diff[diff < 0])
    

In [ ]:
for s in l:
    cval = data.at[0,f'E_{s}']
    shifted = data[f'E_{s}'].shift(1)
    shifted.at[0] = cval
    diff = data[f'E_{s}'] - shifted #each value minus the previous value
    print(f'Instances when E of particle {s} changes:')
    print(diff[diff != 0])
    

In [ ]:
for s in l:
    cval = data.at[0,f'L_{s}']
    shifted = data[f'L_{s}'].shift(1)
    shifted.at[0] = cval
    diff = data[f'L_{s}'] - shifted #each value minus the previous value
    print(f'Instances when L of particle {s} changes:')
    print(diff[diff != 0])
    

In [ ]:
#confirm omega is constant with this result having 0 rows:
data[data['omega'] != data.at[0,'omega']]

In [ ]:
#miscellaneous:

print(data['Ej_14'].max() - data['Ej_14'].min())
print(data.at[0,'m_14'])
print(data['v_14'].mean())
print(data[data['n_14'] != 0])
print(data.at[7189,'t'])

In [ ]:
#how significant is the bar term in Ej?

pl = l
from bokeh.plotting import figure, show, save
from bokeh.models import Legend

TOOLTIPS = [
    ("index", "$index"),
    ("name", "$name"),
    ("(x,y)", "($x, $y)")
]

p = figure(title="other", x_axis_label='x axis', y_axis_label='y axis', sizing_mode = 'stretch_both', tools = "pan,ywheel_pan,tap,box_zoom,wheel_zoom,zoom_in,zoom_out,undo,redo,reset,save", tooltips = TOOLTIPS) #lasso_select,box_select don't seem to work with the lines
from bokeh.palettes import Category20b
import itertools
colours = itertools.cycle(Category20b[20]) #cycle back through list when it reaches end

b = 0.28
a = 235*235
x = numpy.linspace(b,2*b,10000)
ya = a*(0.5 + numpy.log((235/80)*x) - x)
yb = a*0.01*(x**2)*(((b+1)/(b+x))**5)
#for dEj/dr0:
ya = a*((1/x) - 1)
yb = a*0.01*x*(((b+1)/(b+x))**5)*((5/((b/x)+1))-2)
y1 = ya + yb
y2 = ya - yb
plot1 = p.line(x, y1, line_width=2, name = 'y1', selection_alpha = 0.2)
plot2 = p.line(x, y2, line_width=2, name = 'y2', selection_alpha = 0.2)

#show(p)
save(p,'plot2.html')

In [ ]:
dat = {
    'particle': [],
    'initial Ej per unit mass (km^2s^-2)': [],
    'range of Ej per unit mass (km^2s^-2)': [],
    'range of Ej per unit mass (/min)': [],
    'maximum,minimum n': [],
    'index of first non-zero n': [],
    'initial r (Rcr)': [],
    'minimum r (m)': [],
    'maximum a (ms-2)': [],
    'minimum r/a (s-2)': []
}

for s in l:
    dat['particle'].append(s)
    Ejpm = data[f'Ej_{s}']/(1e6)
    ra = data[f'r_{s}']/data[f'a_{s}']
    dat['minimum r/a (s-2)'].append(ra.min())
    dat['initial Ej per unit mass (km^2s^-2)'].append(Ejpm.at[0])
    ran = Ejpm.max() - Ejpm.min()
    dat['range of Ej per unit mass (km^2s^-2)'].append(ran)
    dat['range of Ej per unit mass (/min)'].append(ran/abs(Ejpm.min()))
    dat['maximum,minimum n'].append([data[f'n_{s}'].max(),data[f'n_{s}'].min()])
    #dat['minimum n'].append(data[f'n_{s}'].min())
    r = data[f'r_{s}']/((235/80)*3.086e19)
    dat['initial r (Rcr)'].append(r.at[0])
    dat['minimum r (m)'].append(data[f'r_{s}'].min())
    dat['maximum a (ms-2)'].append(data[f'a_{s}'].max())
    sub = data[data[f'n_{s}'] != 0][f'n_{s}']
    if sub.size != 0:
        idx1 = sub.index[0]
    else:
        idx1 = None
    dat['index of first non-zero n'].append(idx1)

dat1 = pandas.DataFrame(dat)
#dat1.sort_values('range of Ej per unit mass (/min)')
dat1.sort_values('initial Ej per unit mass (km^2s^-2)')

In [ ]:
cols = [col for col in data if '_49' in col]
cols += 't'
data[(data.index > 30810) & (data.index < 30820)][cols]

In [ ]:
#to check the distance between positions of one line of output and the next

cols = [col for s in l for col in [f'xc_{s}',f'yc_{s}']]
#data[cols].iloc[30770:30780]

data1 = data.iloc[30810:30820]
for s in l:
    shifted = data1.shift(1)
    data1['xdif_{s}'] = data1[f'xc_{s}']-shifted[f'xc_{s}']
    data1['ydif_{s}'] = data1[f'yc_{s}']-shifted[f'yc_{s}']
    #print(type(data1['xdif_{s}']))
    data1[f'd2_{s}'] = (data1['xdif_{s}']**2) + (data1['ydif_{s}']**2)
    

cols2 = [col for s in [49] for col in [f'd2_{s}']]
data1[cols2].loc[30810:30820]

In [ ]:
m = data.at[0,f'm_59']
data[f'Ej_59']/(m*1e6)

In [ ]:
np = 1
l = list(range(1,np+1))
#for just a random subset of 20 particles:
#from random import randint
#l = [randint(1,np) for i in range(20)]
#otherwise to load only data of specific particles, make l a list of those particles' numbers
pcols = [col for s in l for col in [f'n_{s}',f'x_{s}',f'y_{s}',f'vx_{s}',f'vy_{s}',f'ax_{s}',f'ay_{s}',f'PE_{s}']]
cols = pcols + ['t', 'phib', 'omega', 'Rcr']
#print(cols)

import time
import pandas
t1 = time.time()
data = pandas.read_csv("C:/data10.txt", usecols=cols, header=0, comment='#')
t2 = time.time()
pandas.set_option('display.max_columns', 500)


import numpy

for s in l:
    data[f'v_{s}'] = numpy.sqrt((data[f'vx_{s}']**2) + (data[f'vy_{s}']**2))
    data[f'a_{s}'] = numpy.sqrt((data[f'ax_{s}']**2) + (data[f'ay_{s}']**2))
    data[f'KE_{s}'] = 0.5*(data[f'v_{s}']**2)
    data[f'r_{s}'] = numpy.sqrt((data[f'x_{s}']**2) + (data[f'y_{s}']**2))
    data[f'phi_{s}'] = numpy.arctan2(data[f'y_{s}'], data[f'x_{s}'])
    data[f'phic_{s}'] = data[f'phi_{s}'] - data['phib']
    data[f'xc_{s}'] = data[f'r_{s}']*numpy.cos(data[f'phic_{s}'])
    data[f'yc_{s}'] = data[f'r_{s}']*numpy.sin(data[f'phic_{s}'])
    data[f'vr_{s}'] = (1/data[f'r_{s}'])*((data[f'vy_{s}']*data[f'y_{s}']) + (data[f'vx_{s}']*data[f'x_{s}']))
    data[f'vphi_{s}'] = (1/data[f'r_{s}'])*((data[f'vy_{s}']*data[f'x_{s}']) - (data[f'vx_{s}']*data[f'y_{s}']))
    data[f'phidot_{s}'] = data[f'vphi_{s}']/data[f'r_{s}']
    data[f'phidotc_{s}'] = data[f'phidot_{s}'] - data['omega']
    data[f'vxc_{s}'] = data[f'vr_{s}']*numpy.cos(data[f'phic_{s}']) - data[f'r_{s}']*data[f'phidotc_{s}']*numpy.sin(data[f'phic_{s}'])
    data[f'vyc_{s}'] = data[f'vr_{s}']*numpy.sin(data[f'phic_{s}']) + data[f'r_{s}']*data[f'phidotc_{s}']*numpy.cos(data[f'phic_{s}'])
    data[f'L_{s}'] = data[f'r_{s}']*data[f'vphi_{s}']
    data[f'E_{s}'] = data[f'KE_{s}'] + data[f'PE_{s}']
    data[f'Ej_{s}'] = data[f'E_{s}'] - (data['omega']*data[f'L_{s}'])
    
    
dat = {
    'particle': [],
    'initial Ej per unit mass (km^2s^-2)': [],
    'range of Ej per unit mass (km^2s^-2)': [],
    'range of Ej per unit mass (/min)': [],
    'maximum n': [],
    'index when n first increases from 0': [],
    'initial r (Rcr)': [],
    'minimum r (m)': [],
    'maximum a (ms-2)': [],
    'minimum r/a (s-2)': []
}

for s in l:
    dat['particle'].append(s)
    Ejpm = data[f'Ej_{s}']/(1e6)
    ra = data[f'r_{s}']/data[f'a_{s}']
    dat['minimum r/a (s-2)'].append(ra.min())
    dat['initial Ej per unit mass (km^2s^-2)'].append(Ejpm.at[0])

    ran = Ejpm.max() - Ejpm.min()
    dat['range of Ej per unit mass (km^2s^-2)'].append(ran)
    dat['range of Ej per unit mass (/min)'].append(ran/abs(Ejpm.min()))
    dat['maximum n'].append(data[f'n_{s}'].max())
    r = data[f'r_{s}']/((235/80)*3.086e19)
    dat['initial r (Rcr)'].append(r.at[0])
    dat['minimum r (m)'].append(data[f'r_{s}'].min())
    dat['maximum a (ms-2)'].append(data[f'a_{s}'].max())
    sub = data[data[f'n_{s}'] != 0][f'n_{s}']
    if sub.size != 0:
        idx1 = sub.index[0]
    else:
        idx1 = None
    dat['index when n first increases from 0'].append(idx1)

dat1 = pandas.DataFrame(dat)
#dat1.sort_values('range of Ej per unit mass (/min)')
dat1.sort_values('initial Ej per unit mass (km^2s^-2)')

In [ ]:
with open('data.txt','a') as file:
    print(3.93e-3,1.23,dat1['minimum r (m)'].at[0], dat1['maximum a (ms-2)'].at[0], dat1['minimum r/a (s-2)'].at[0],file=file,sep=',')

In [ ]:
cdata = pandas.read_csv("C:/data.txt", header=0, comment='#')
cdata=cdata.sort_values(by=['initial r (b*Rcr)'])
display(cdata)
print(cdata['maximum a (ms-2)'].iat[0])
from bokeh.plotting import figure, show, save
from bokeh.models import Legend

TOOLTIPS = [
    ("index", "$index"),
    ("name", "$name"),
    ("(x,y)", "($x, $y)")
]

p = figure(title="other", x_axis_label='x axis', y_axis_label='y axis', sizing_mode = 'stretch_both', tools = "pan,ywheel_pan,tap,box_zoom,wheel_zoom,zoom_in,zoom_out,undo,redo,reset,save", tooltips = TOOLTIPS) #lasso_select,box_select don't seem to work with the lines

y = numpy.log2(cdata['dtmax (Myr)'])
x = cdata['maximum a (ms-2)']
p.line(x,y,name='data')
x1 = numpy.linspace(cdata['maximum a (ms-2)'].iat[0],cdata['maximum a (ms-2)'].iat[-1],10000)
pow = 2
k = 0.1*3.154e13*(7.18e-19**pow)
y1 = numpy.log2(k/(x**pow))
p.line(x,y1,name='fit')
print(y1-y < 0)

show(p)